# Modern CNN Architectures: From ResNet to Vision Transformers

**Building State-of-the-Art Computer Vision Models with PyTorch**

**Authors:** PyTorch Mastery Hub Team  
**Institution:** Advanced Deep Learning Research  
**Course:** Computer Vision and Deep Learning  

## Overview

This notebook provides comprehensive implementation and analysis of modern CNN architectures that revolutionized computer vision. We explore the evolution from traditional CNNs to cutting-edge Vision Transformers, implementing each architecture from scratch and conducting thorough performance comparisons.

## Key Objectives
1. Implement ResNet from scratch with skip connections and residual learning
2. Build DenseNet with dense connectivity and feature reuse mechanisms
3. Create EfficientNet with compound scaling and mobile optimization
4. Develop Vision Transformer fundamentals with self-attention mechanisms
5. Conduct comprehensive architecture comparisons and performance analysis
6. Execute advanced transfer learning experiments with different strategies
7. Generate professional visualizations and analysis reports

## 1. Environment Setup and Configuration

In [ ]:
# Core PyTorch and Computer Vision Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

# Scientific Computing and Visualization
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import time
from datetime import datetime
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Advanced Analysis Tools
try:
    from torchinfo import summary
except ImportError:
    print("⚠️ torchinfo not available. Install with: pip install torchinfo")
    def summary(*args, **kwargs):
        return "Summary not available"

from collections import OrderedDict
import math

# Configuration and Styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Professional Directory Structure
def setup_comprehensive_directories():
    """Create comprehensive directory structure for modern CNN architectures project"""
    base_dirs = [
        "../results/modern_cnn_architectures/analysis/architecture_comparisons",
        "../results/modern_cnn_architectures/analysis/performance_benchmarks", 
        "../results/modern_cnn_architectures/experiments/transfer_learning",
        "../results/modern_cnn_architectures/experiments/attention_visualization",
        "../results/modern_cnn_architectures/experiments/scaling_analysis",
        "../results/modern_cnn_architectures/visualizations/model_structures",
        "../results/modern_cnn_architectures/visualizations/training_progress",
        "../models/modern_cnn_architectures/pretrained",
        "../models/modern_cnn_architectures/custom_trained",
        "../data/modern_cnn_architectures/processed"
    ]
    
    created_dirs = {}
    for dir_path in base_dirs:
        Path(dir_path).mkdir(parents=True, exist_ok=True)
        dir_name = Path(dir_path).name
        created_dirs[dir_name] = dir_path
        print(f"📁 Created: {dir_path}")
    
    return created_dirs

# Initialize directory structure
project_dirs = setup_comprehensive_directories()
print(f"\n✅ Comprehensive directory structure initialized!")
print(f"📊 Results will be saved to: ../results/modern_cnn_architectures/")
print(f"💾 Models will be saved to: ../models/modern_cnn_architectures/")

# Utility classes for benchmarking and analysis
class Timer:
    """Professional timer for benchmarking operations"""
    def __init__(self):
        self.start_time = None
        self.history = []
    
    def start(self):
        self.start_time = time.time()
    
    def stop(self):
        if self.start_time is None:
            return 0
        elapsed = time.time() - self.start_time
        self.history.append(elapsed)
        self.start_time = None
        return elapsed
    
    def average(self):
        return np.mean(self.history) if self.history else 0

class ModelAnalyzer:
    """Comprehensive model analysis utilities"""
    
    @staticmethod
    def count_parameters(model):
        """Count total and trainable parameters"""
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        return total_params, trainable_params
    
    @staticmethod
    def calculate_model_size(model):
        """Calculate model size in MB"""
        total_params, _ = ModelAnalyzer.count_parameters(model)
        # Assuming float32 (4 bytes per parameter)
        size_mb = total_params * 4 / (1024 ** 2)
        return size_mb
    
    @staticmethod
    def get_layer_info(model):
        """Extract detailed layer information"""
        layer_info = []
        for name, module in model.named_modules():
            if len(list(module.children())) == 0:  # Leaf modules only
                layer_info.append({
                    'name': name,
                    'type': type(module).__name__,
                    'parameters': sum(p.numel() for p in module.parameters())
                })
        return layer_info

print("\n🔧 Analysis utilities initialized successfully!")

## 2. ResNet Implementation: Skip Connections and Residual Learning

In [ ]:
class BasicResNetBlock(nn.Module):
    """
    Basic ResNet block implementing residual connections for gradient flow improvement
    
    This block solves the vanishing gradient problem through skip connections,
    enabling training of very deep networks.
    """
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicResNetBlock, self).__init__()
        
        # First convolution with potential stride for downsampling
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # Second convolution (always stride 1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Downsample layer for dimension matching
        self.downsample = downsample
        self.stride = stride
        
        # Track forward pass statistics
        self.forward_count = 0
    
    def forward(self, x):
        self.forward_count += 1
        identity = x
        
        # First convolution block
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)
        
        # Second convolution block
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Apply downsampling to identity if dimensions don't match
        if self.downsample is not None:
            identity = self.downsample(x)
        
        # The key innovation: residual connection
        out += identity
        out = F.relu(out, inplace=True)
        
        return out

class BottleneckResNetBlock(nn.Module):
    """
    Bottleneck ResNet block for deeper networks (ResNet-50+)
    
    Uses 1x1 convolutions to reduce computational complexity while maintaining
    representational power through bottleneck architecture.
    """
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BottleneckResNetBlock, self).__init__()
        
        # 1x1 convolution for dimension reduction
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # 3x3 convolution (main computation)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # 1x1 convolution for dimension expansion
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion,
                              kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        
        self.downsample = downsample
        self.stride = stride
    
    def forward(self, x):
        identity = x
        
        # Bottleneck: reduce → process → expand
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        out = self.bn3(self.conv3(out))
        
        # Skip connection
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = F.relu(out, inplace=True)
        
        return out

class ResNetArchitecture(nn.Module):
    """
    Complete ResNet implementation with comprehensive analysis capabilities
    
    Supports multiple ResNet variants (18, 34, 50, 101, 152) through
    configurable block arrangements and types.
    """
    
    def __init__(self, block, layers, num_classes=10, input_channels=3, 
                 zero_init_residual=False):
        super(ResNetArchitecture, self).__init__()
        
        self.in_channels = 64
        self.block_type = block.__name__
        self.layer_config = layers
        
        # Initial convolution and pooling
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, 
                              padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers with progressive channel increase
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        # Global average pooling and classification
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # Initialize weights with modern best practices
        self._initialize_weights(zero_init_residual)
        
        # Analysis attributes
        self.feature_maps = {}
        self.gradient_flows = {}
    
    def _make_layer(self, block, out_channels, blocks, stride=1):
        """Create a residual layer with multiple blocks"""
        downsample = None
        
        # Create downsampling layer if needed for dimension matching
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                         kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )
        
        layers = []
        # First block handles stride and potential downsampling
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        
        # Remaining blocks maintain dimensions
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def _initialize_weights(self, zero_init_residual):
        """Initialize weights using He initialization for ReLU networks"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        # Zero-initialize last BN in residual branches for better training
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BottleneckResNetBlock):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicResNetBlock):
                    nn.init.constant_(m.bn2.weight, 0)
    
    def forward(self, x):
        """Forward pass with optional feature extraction"""
        # Initial processing
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x, inplace=True)
        x = self.maxpool(x)
        
        # Progressive residual processing
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Global pooling and classification
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
    
    def extract_features(self, x, layer_names=None):
        """Extract intermediate feature maps for analysis"""
        features = {}
        
        # Initial layers
        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        features['conv1'] = x.clone().detach()
        x = self.maxpool(x)
        features['maxpool'] = x.clone().detach()
        
        # Residual layers
        x = self.layer1(x)
        features['layer1'] = x.clone().detach()
        x = self.layer2(x)
        features['layer2'] = x.clone().detach()
        x = self.layer3(x)
        features['layer3'] = x.clone().detach()
        x = self.layer4(x)
        features['layer4'] = x.clone().detach()
        
        return features
    
    def get_architecture_info(self):
        """Get comprehensive architecture information"""
        total_params, trainable_params = ModelAnalyzer.count_parameters(self)
        model_size = ModelAnalyzer.calculate_model_size(self)
        
        return {
            'architecture': 'ResNet',
            'block_type': self.block_type,
            'layer_config': self.layer_config,
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'model_size_mb': model_size,
            'depth': sum(self.layer_config) * 2 + 2  # Approximate depth
        }

# Factory functions for different ResNet variants
def create_resnet18(num_classes=10, input_channels=3):
    """Create ResNet-18 with 18 layers"""
    return ResNetArchitecture(BasicResNetBlock, [2, 2, 2, 2], num_classes, input_channels)

def create_resnet34(num_classes=10, input_channels=3):
    """Create ResNet-34 with 34 layers"""
    return ResNetArchitecture(BasicResNetBlock, [3, 4, 6, 3], num_classes, input_channels)

def create_resnet50(num_classes=10, input_channels=3):
    """Create ResNet-50 with bottleneck blocks"""
    return ResNetArchitecture(BottleneckResNetBlock, [3, 4, 6, 3], num_classes, input_channels)

def create_resnet101(num_classes=10, input_channels=3):
    """Create ResNet-101 for very deep learning"""
    return ResNetArchitecture(BottleneckResNetBlock, [3, 4, 23, 3], num_classes, input_channels)

# Initialize ResNet model collection
print("🔗 Creating ResNet Architecture Collection:")
resnet_models = {
    'ResNet-18': create_resnet18(),
    'ResNet-34': create_resnet34(),
    'ResNet-50': create_resnet50()
}

# Store comprehensive model information
models_info = {}
for name, model in resnet_models.items():
    model = model.to(device)
    info = model.get_architecture_info()
    models_info[name] = info
    
    print(f"\n📊 {name} Analysis:")
    print(f"   Parameters: {info['total_parameters']:,}")
    print(f"   Size: {info['model_size_mb']:.2f} MB")
    print(f"   Depth: {info['depth']} layers")
    print(f"   Block type: {info['block_type']}")

print(f"\n✅ ResNet collection initialized with {len(resnet_models)} variants!")

## 3. DenseNet Implementation: Dense Connectivity and Feature Reuse

In [ ]:
class DenseLayer(nn.Module):
    """
    Dense layer implementing feature concatenation for maximum information flow
    
    Each layer receives feature maps from ALL preceding layers, enabling
    feature reuse and reducing the number of parameters needed.
    """
    
    def __init__(self, in_channels, growth_rate, bn_size=4, drop_rate=0.1):
        super(DenseLayer, self).__init__()
        
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate
        
        # Bottleneck layers for computational efficiency
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, bn_size * growth_rate, 
                              kernel_size=1, bias=False)
        
        self.bn2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate,
                              kernel_size=3, padding=1, bias=False)
        
        # Track concatenation statistics
        self.concat_count = 0
    
    def forward(self, x):
        # Pre-activation design: BN → ReLU → Conv
        bottleneck = self.conv1(F.relu(self.bn1(x), inplace=True))
        new_features = self.conv2(F.relu(self.bn2(bottleneck), inplace=True))
        
        # Apply dropout for regularization
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        
        # The key innovation: concatenate with input (dense connection)
        self.concat_count += 1
        return torch.cat([x, new_features], 1)

class DenseBlock(nn.Module):
    """
    Dense block containing multiple dense layers with progressive feature growth
    
    Features grow by growth_rate at each layer, creating rich feature representations
    through dense connectivity patterns.
    """
    
    def __init__(self, num_layers, in_channels, growth_rate, bn_size=4, drop_rate=0.1):
        super(DenseBlock, self).__init__()
        
        self.num_layers = num_layers
        self.growth_rate = growth_rate
        
        # Create sequence of dense layers
        layers = []
        for i in range(num_layers):
            layer = DenseLayer(
                in_channels + i * growth_rate, 
                growth_rate, 
                bn_size, 
                drop_rate
            )
            layers.append(layer)
        
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        # Progressive feature concatenation
        for layer in self.layers:
            x = layer(x)
        return x
    
    def get_output_channels(self, in_channels):
        """Calculate output channels after dense block"""
        return in_channels + self.num_layers * self.growth_rate

class DenseTransition(nn.Module):
    """
    Transition layer between dense blocks for dimension reduction
    
    Reduces feature map size and number of channels to control model complexity
    and memory usage between dense blocks.
    """
    
    def __init__(self, in_channels, out_channels, compression_factor=0.5):
        super(DenseTransition, self).__init__()
        
        self.compression_factor = compression_factor
        
        # Dimension reduction through 1x1 convolution
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        out = self.conv(F.relu(self.bn(x), inplace=True))
        out = self.pool(out)
        return out

class DenseNetArchitecture(nn.Module):
    """
    Complete DenseNet implementation with comprehensive feature analysis
    
    Implements dense connectivity where each layer connects to every other layer
    in a feed-forward fashion, maximizing information flow and gradient propagation.
    """
    
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0.1, 
                 num_classes=10, input_channels=3, compression_factor=0.5):
        super(DenseNetArchitecture, self).__init__()
        
        self.growth_rate = growth_rate
        self.block_config = block_config
        self.compression_factor = compression_factor
        
        # Initial convolution and pooling
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, num_init_features, kernel_size=7, 
                     stride=2, padding=3, bias=False),
            nn.BatchNorm2d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # Progressive dense blocks with transitions
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            # Dense block
            block = DenseBlock(num_layers, num_features, growth_rate, bn_size, drop_rate)
            self.features.add_module(f'denseblock{i+1}', block)
            num_features = block.get_output_channels(num_features)
            
            # Transition layer (except for last block)
            if i != len(block_config) - 1:
                trans_channels = int(num_features * compression_factor)
                trans = DenseTransition(num_features, trans_channels, compression_factor)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = trans_channels
        
        # Final batch normalization
        self.features.add_module('norm_final', nn.BatchNorm2d(num_features))
        
        # Classification head
        self.classifier = nn.Linear(num_features, num_classes)
        
        # Initialize weights
        self._initialize_weights()
        
        # Store architecture info
        self.final_feature_size = num_features
    
    def _initialize_weights(self):
        """Initialize weights for optimal convergence"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out
    
    def extract_dense_features(self, x):
        """Extract features from each dense block for analysis"""
        features = {}
        
        # Initial features
        x = self.features.conv1(x)
        x = self.features.bn1(x)
        x = F.relu(x, inplace=True)
        features['initial'] = x.clone().detach()
        x = self.features.maxpool(x)
        
        # Extract from each dense block
        for name, module in self.features.named_children():
            if 'denseblock' in name:
                x = module(x)
                features[name] = x.clone().detach()
            elif 'transition' in name:
                x = module(x)
                features[name] = x.clone().detach()
        
        return features
    
    def get_architecture_info(self):
        """Get comprehensive DenseNet architecture information"""
        total_params, trainable_params = ModelAnalyzer.count_parameters(self)
        model_size = ModelAnalyzer.calculate_model_size(self)
        
        return {
            'architecture': 'DenseNet',
            'growth_rate': self.growth_rate,
            'block_config': self.block_config,
            'compression_factor': self.compression_factor,
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'model_size_mb': model_size,
            'final_feature_size': self.final_feature_size
        }

# Factory functions for different DenseNet variants
def create_densenet121(num_classes=10, input_channels=3):
    """Create DenseNet-121 with efficient architecture"""
    return DenseNetArchitecture(
        growth_rate=32, 
        block_config=(6, 12, 24, 16), 
        num_init_features=64,
        num_classes=num_classes, 
        input_channels=input_channels
    )

def create_densenet169(num_classes=10, input_channels=3):
    """Create DenseNet-169 with deeper blocks"""
    return DenseNetArchitecture(
        growth_rate=32, 
        block_config=(6, 12, 32, 32),
        num_init_features=64,
        num_classes=num_classes, 
        input_channels=input_channels
    )

def create_densenet201(num_classes=10, input_channels=3):
    """Create DenseNet-201 with maximum depth"""
    return DenseNetArchitecture(
        growth_rate=32, 
        block_config=(6, 12, 48, 32),
        num_init_features=64,
        num_classes=num_classes, 
        input_channels=input_channels
    )

# Initialize DenseNet model collection
print("\n🌟 Creating DenseNet Architecture Collection:")
densenet_models = {
    'DenseNet-121': create_densenet121(),
    'DenseNet-169': create_densenet169()
}

for name, model in densenet_models.items():
    model = model.to(device)
    info = model.get_architecture_info()
    models_info[name] = info
    
    print(f"\n📊 {name} Analysis:")
    print(f"   Parameters: {info['total_parameters']:,}")
    print(f"   Size: {info['model_size_mb']:.2f} MB")
    print(f"   Growth rate: {info['growth_rate']}")
    print(f"   Block config: {info['block_config']}")
    print(f"   Compression factor: {info['compression_factor']}")

print(f"\n✅ DenseNet collection initialized with {len(densenet_models)} variants!")

## 4. EfficientNet Implementation: Compound Scaling and Mobile Optimization

In [ ]:
class SqueezeExcitationBlock(nn.Module):
    """
    Squeeze-and-Excitation block for channel attention mechanism
    
    Implements adaptive recalibration of channel-wise feature responses
    by explicitly modelling interdependencies between channels.
    """
    
    def __init__(self, in_channels, reduction_ratio=16):
        super(SqueezeExcitationBlock, self).__init__()
        
        reduced_channels = max(1, in_channels // reduction_ratio)
        
        # Global average pooling for squeeze operation
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # Excitation mechanism with dimensionality reduction and expansion
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, reduced_channels, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(reduced_channels, in_channels, bias=False),
            nn.Sigmoid()
        )
        
        # Track attention statistics
        self.attention_weights = None
    
    def forward(self, x):
        batch_size, channels, _, _ = x.size()
        
        # Squeeze: Global spatial information compression
        squeeze = self.avg_pool(x).view(batch_size, channels)
        
        # Excitation: Channel importance weights
        excitation = self.excitation(squeeze).view(batch_size, channels, 1, 1)
        self.attention_weights = excitation.clone().detach()
        
        # Scale: Apply attention weights
        return x * excitation.expand_as(x)

class MobileInvertedBottleneckBlock(nn.Module):
    """
    Mobile Inverted Bottleneck Convolution block with efficiency optimizations
    
    Key innovations:
    - Depthwise separable convolutions for efficiency
    - Inverted residuals for better gradient flow
    - Linear bottlenecks to preserve information
    """
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, 
                 expand_ratio, se_ratio=0.25, drop_rate=0.1):
        super(MobileInvertedBottleneckBlock, self).__init__()
        
        self.stride = stride
        self.drop_rate = drop_rate
        self.use_residual = stride == 1 and in_channels == out_channels
        self.expand_ratio = expand_ratio
        
        # Expansion phase (inverted residual)
        expanded_channels = in_channels * expand_ratio
        
        if expand_ratio != 1:
            self.expand_conv = nn.Sequential(
                nn.Conv2d(in_channels, expanded_channels, 1, bias=False),
                nn.BatchNorm2d(expanded_channels),
                nn.ReLU6(inplace=True)
            )
        else:
            self.expand_conv = nn.Identity()
        
        # Depthwise convolution for spatial processing
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(expanded_channels, expanded_channels, kernel_size, stride, 
                     kernel_size//2, groups=expanded_channels, bias=False),
            nn.BatchNorm2d(expanded_channels),
            nn.ReLU6(inplace=True)
        )
        
        # Squeeze-and-Excitation for channel attention
        if se_ratio > 0:
            se_channels = max(1, int(in_channels * se_ratio))
            self.se = SqueezeExcitationBlock(expanded_channels, 
                                           expanded_channels // se_channels)
        else:
            self.se = nn.Identity()
        
        # Pointwise convolution for channel mixing (linear bottleneck)
        self.pointwise_conv = nn.Sequential(
            nn.Conv2d(expanded_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels)
            # Note: No activation here (linear bottleneck)
        )
        
        # Performance tracking
        self.forward_count = 0
    
    def forward(self, x):
        self.forward_count += 1
        identity = x
        
        # Expansion phase
        x = self.expand_conv(x)
        
        # Depthwise spatial processing
        x = self.depthwise_conv(x)
        
        # Channel attention
        x = self.se(x)
        
        # Pointwise channel mixing (linear bottleneck)
        x = self.pointwise_conv(x)
        
        # Residual connection with stochastic depth
        if self.use_residual:
            if self.drop_rate > 0 and self.training:
                x = F.dropout(x, p=self.drop_rate, training=self.training)
            x = x + identity
        
        return x

class EfficientNetArchitecture(nn.Module):
    """
    EfficientNet implementation with compound scaling methodology
    
    Systematically scales network depth, width, and resolution using
    compound scaling coefficients for optimal accuracy-efficiency trade-offs.
    """
    
    def __init__(self, width_mult=1.0, depth_mult=1.0, resolution=224, 
                 num_classes=10, input_channels=3, drop_rate=0.2, 
                 stochastic_depth_rate=0.2):
        super(EfficientNetArchitecture, self).__init__()
        
        self.width_mult = width_mult
        self.depth_mult = depth_mult
        self.resolution = resolution
        self.drop_rate = drop_rate
        
        # Base configuration for EfficientNet-B0
        # (expand_ratio, channels, num_layers, stride, kernel_size)
        base_config = [
            (1, 16, 1, 1, 3),   # Stage 1
            (6, 24, 2, 2, 3),   # Stage 2
            (6, 40, 2, 2, 5),   # Stage 3
            (6, 80, 3, 2, 3),   # Stage 4
            (6, 112, 3, 1, 5),  # Stage 5
            (6, 192, 4, 2, 5),  # Stage 6
            (6, 320, 1, 1, 3),  # Stage 7
        ]
        
        # Apply compound scaling to base configuration
        def round_filters(filters, width_mult):
            """Round number of filters based on width multiplier"""
            if width_mult == 1.0:
                return filters
            filters *= width_mult
            new_filters = max(8, int(filters + 4) // 8 * 8)
            if new_filters < 0.9 * filters:
                new_filters += 8
            return int(new_filters)
        
        def round_repeats(repeats, depth_mult):
            """Round number of layer repeats based on depth multiplier"""
            if depth_mult == 1.0:
                return repeats
            return int(math.ceil(depth_mult * repeats))
        
        # Stem convolution
        out_channels = round_filters(32, width_mult)
        self.stem = nn.Sequential(
            nn.Conv2d(input_channels, out_channels, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6(inplace=True)
        )
        
        # Build inverted bottleneck blocks
        features = []
        in_channels = out_channels
        total_blocks = sum(round_repeats(layers, depth_mult) for _, _, layers, _, _ in base_config)
        block_idx = 0
        
        for expand_ratio, channels, num_layers, stride, kernel_size in base_config:
            out_channels = round_filters(channels, width_mult)
            num_layers = round_repeats(num_layers, depth_mult)
            
            for i in range(num_layers):
                # Stochastic depth probability increases linearly
                drop_path_rate = stochastic_depth_rate * block_idx / total_blocks
                
                features.append(MobileInvertedBottleneckBlock(
                    in_channels, out_channels, kernel_size,
                    stride if i == 0 else 1, expand_ratio,
                    se_ratio=0.25, drop_rate=drop_path_rate
                ))
                in_channels = out_channels
                block_idx += 1
        
        self.features = nn.Sequential(*features)
        
        # Head: final convolution and classification
        last_channels = round_filters(1280, width_mult)
        self.head = nn.Sequential(
            nn.Conv2d(in_channels, last_channels, 1, bias=False),
            nn.BatchNorm2d(last_channels),
            nn.ReLU6(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(drop_rate) if drop_rate > 0 else nn.Identity(),
        )
        
        self.classifier = nn.Linear(last_channels, num_classes)
        
        # Initialize weights
        self._initialize_weights()
        
        # Store scaling information
        self.last_channels = last_channels
        self.total_blocks = total_blocks
    
    def _initialize_weights(self):
        """Initialize weights for efficient training"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.head(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
    def extract_efficient_features(self, x):
        """Extract features at multiple scales for analysis"""
        features = {}
        
        # Stem
        x = self.stem(x)
        features['stem'] = x.clone().detach()
        
        # Sample features from different stages
        stage_indices = [0, 3, 6, 10, 16, 23, 30]  # Approximate stage boundaries
        
        for i, layer in enumerate(self.features):
            x = layer(x)
            if i in stage_indices:
                features[f'stage_{stage_indices.index(i)}'] = x.clone().detach()
        
        return features
    
    def get_architecture_info(self):
        """Get comprehensive EfficientNet architecture information"""
        total_params, trainable_params = ModelAnalyzer.count_parameters(self)
        model_size = ModelAnalyzer.calculate_model_size(self)
        
        return {
            'architecture': 'EfficientNet',
            'width_multiplier': self.width_mult,
            'depth_multiplier': self.depth_mult,
            'resolution': self.resolution,
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'model_size_mb': model_size,
            'total_blocks': self.total_blocks,
            'final_channels': self.last_channels
        }

# Factory functions for different EfficientNet variants
def create_efficientnet_b0(num_classes=10, input_channels=3):
    """Create EfficientNet-B0 baseline model"""
    return EfficientNetArchitecture(
        width_mult=1.0, depth_mult=1.0, resolution=224,
        num_classes=num_classes, input_channels=input_channels
    )

def create_efficientnet_b1(num_classes=10, input_channels=3):
    """Create EfficientNet-B1 with light scaling"""
    return EfficientNetArchitecture(
        width_mult=1.0, depth_mult=1.1, resolution=240,
        num_classes=num_classes, input_channels=input_channels
    )

def create_efficientnet_b2(num_classes=10, input_channels=3):
    """Create EfficientNet-B2 with moderate scaling"""
    return EfficientNetArchitecture(
        width_mult=1.1, depth_mult=1.2, resolution=260,
        num_classes=num_classes, input_channels=input_channels
    )

# Initialize EfficientNet model collection
print("\n⚡ Creating EfficientNet Architecture Collection:")
efficientnet_models = {
    'EfficientNet-B0': create_efficientnet_b0(),
    'EfficientNet-B1': create_efficientnet_b1()
}

for name, model in efficientnet_models.items():
    model = model.to(device)
    info = model.get_architecture_info()
    models_info[name] = info
    
    print(f"\n📊 {name} Analysis:")
    print(f"   Parameters: {info['total_parameters']:,}")
    print(f"   Size: {info['model_size_mb']:.2f} MB")
    print(f"   Width multiplier: {info['width_multiplier']}")
    print(f"   Depth multiplier: {info['depth_multiplier']}")
    print(f"   Total blocks: {info['total_blocks']}")

print(f"\n✅ EfficientNet collection initialized with {len(efficientnet_models)} variants!")

## 5. Vision Transformer Implementation: Self-Attention for Computer Vision

In [ ]:
class PatchEmbeddingLayer(nn.Module):
    """
    Convert image patches to embeddings for transformer processing
    
    Divides input image into non-overlapping patches and linearly embeds
    each patch, treating them as tokens for the transformer.
    """
    
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=192):
        super(PatchEmbeddingLayer, self).__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.embed_dim = embed_dim
        
        # Convolutional layer for patch extraction and embedding
        self.projection = nn.Conv2d(
            in_channels, embed_dim, 
            kernel_size=patch_size, stride=patch_size
        )
        
        # Layer normalization for stable training
        self.norm = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        # x: (Batch, Channels, Height, Width)
        batch_size = x.shape[0]
        
        # Extract patches and embed: (B, embed_dim, H/patch_size, W/patch_size)
        x = self.projection(x)
        
        # Flatten spatial dimensions: (B, embed_dim, num_patches)
        x = x.flatten(2)
        
        # Transpose for transformer: (B, num_patches, embed_dim)
        x = x.transpose(1, 2)
        
        # Apply layer normalization
        x = self.norm(x)
        
        return x

class MultiHeadSelfAttentionBlock(nn.Module):
    """
    Multi-head self-attention mechanism for capturing global dependencies
    
    Enables the model to attend to different parts of the input sequence
    simultaneously through multiple attention heads.
    """
    
    def __init__(self, embed_dim, num_heads, dropout=0.1, qkv_bias=True):
        super(MultiHeadSelfAttentionBlock, self).__init__()
        
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        
        # Linear projections for queries, keys, and values
        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=qkv_bias)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        # Attention weights for visualization
        self.attention_weights = None
    
    def forward(self, x):
        batch_size, seq_len, embed_dim = x.shape
        
        # Generate Q, K, V matrices
        qkv = self.qkv(x).reshape(batch_size, seq_len, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, B, num_heads, seq_len, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        # Scaled dot-product attention
        attention_scores = (q @ k.transpose(-2, -1)) * self.scale
        attention_weights = attention_scores.softmax(dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Store attention weights for visualization
        self.attention_weights = attention_weights.clone().detach()
        
        # Apply attention to values
        attention_output = (attention_weights @ v).transpose(1, 2).reshape(
            batch_size, seq_len, embed_dim
        )
        
        # Final projection
        output = self.proj(attention_output)
        output = self.dropout(output)
        
        return output

class TransformerMLPBlock(nn.Module):
    """
    MLP block with GELU activation for transformer layers
    
    Provides non-linear transformation with expansion and contraction
    for enhanced representational capacity.
    """
    
    def __init__(self, embed_dim, mlp_ratio=4.0, dropout=0.1):
        super(TransformerMLPBlock, self).__init__()
        
        hidden_dim = int(embed_dim * mlp_ratio)
        
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.activation = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class TransformerEncoderBlock(nn.Module):
    """
    Complete transformer encoder block with attention and MLP
    
    Implements the standard transformer architecture with:
    - Multi-head self-attention
    - Layer normalization
    - MLP with residual connections
    """
    
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super(TransformerEncoderBlock, self).__init__()
        
        # Pre-normalization design for better gradient flow
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attention = MultiHeadSelfAttentionBlock(embed_dim, num_heads, dropout)
        
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = TransformerMLPBlock(embed_dim, mlp_ratio, dropout)
        
        # Track block statistics
        self.forward_count = 0
    
    def forward(self, x):
        self.forward_count += 1
        
        # Self-attention with residual connection (pre-norm)
        attention_output = self.attention(self.norm1(x))
        x = x + attention_output
        
        # MLP with residual connection (pre-norm)
        mlp_output = self.mlp(self.norm2(x))
        x = x + mlp_output
        
        return x

class VisionTransformerArchitecture(nn.Module):
    """
    Complete Vision Transformer implementation for image classification
    
    Adapts the transformer architecture for computer vision by:
    - Converting images to patch sequences
    - Adding learnable position embeddings
    - Using a class token for classification
    """
    
    def __init__(self, img_size=32, patch_size=4, in_channels=3, num_classes=10,
                 embed_dim=192, depth=12, num_heads=3, mlp_ratio=4.0, 
                 dropout=0.1, attention_dropout=0.1):
        super(VisionTransformerArchitecture, self).__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.depth = depth
        self.num_heads = num_heads
        
        # Patch embedding
        self.patch_embed = PatchEmbeddingLayer(img_size, patch_size, in_channels, embed_dim)
        num_patches = self.patch_embed.num_patches
        
        # Learnable parameters
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_dropout = nn.Dropout(dropout)
        
        # Transformer encoder blocks
        self.transformer_blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_ratio, attention_dropout)
            for _ in range(depth)
        ])
        
        # Classification head
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)
        
        # Initialize weights
        self._initialize_weights()
        
        # Store architecture info
        self.num_patches = num_patches
    
    def _initialize_weights(self):
        """Initialize transformer weights using appropriate distributions"""
        # Initialize position embeddings and class token
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        
        # Initialize other weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LayerNorm):
                nn.init.constant_(m.bias, 0)
                nn.init.constant_(m.weight, 1.0)
            elif isinstance(m, nn.Conv2d):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Convert image to patch embeddings
        x = self.patch_embed(x)  # (B, num_patches, embed_dim)
        
        # Add class token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)  # (B, num_patches + 1, embed_dim)
        
        # Add position embeddings
        x = x + self.pos_embed
        x = self.pos_dropout(x)
        
        # Pass through transformer blocks
        attention_weights = []
        for block in self.transformer_blocks:
            x = block(x)
            if hasattr(block.attention, 'attention_weights') and block.attention.attention_weights is not None:
                attention_weights.append(block.attention.attention_weights)
        
        # Apply final layer norm
        x = self.norm(x)
        
        # Classification using class token
        cls_token_final = x[:, 0]  # First token is class token
        logits = self.head(cls_token_final)
        
        return logits, attention_weights
    
    def extract_transformer_features(self, x):
        """Extract features from different transformer layers"""
        features = {}
        batch_size = x.shape[0]
        
        # Patch embedding
        x = self.patch_embed(x)
        features['patch_embed'] = x.clone().detach()
        
        # Add tokens and positions
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_dropout(x)
        features['input_with_pos'] = x.clone().detach()
        
        # Extract from transformer blocks
        for i, block in enumerate(self.transformer_blocks):
            x = block(x)
            if i % 3 == 0:  # Sample every 3rd layer
                features[f'transformer_block_{i}'] = x.clone().detach()
        
        return features
    
    def get_architecture_info(self):
        """Get comprehensive Vision Transformer architecture information"""
        total_params, trainable_params = ModelAnalyzer.count_parameters(self)
        model_size = ModelAnalyzer.calculate_model_size(self)
        
        return {
            'architecture': 'Vision Transformer',
            'img_size': self.img_size,
            'patch_size': self.patch_size,
            'embed_dim': self.embed_dim,
            'depth': self.depth,
            'num_heads': self.num_heads,
            'num_patches': self.num_patches,
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'model_size_mb': model_size
        }

# Factory functions for different ViT variants
def create_vit_tiny(num_classes=10, input_channels=3):
    """Create ViT-Tiny for efficient processing"""
    return VisionTransformerArchitecture(
        img_size=32, patch_size=4, in_channels=input_channels, num_classes=num_classes,
        embed_dim=192, depth=12, num_heads=3, mlp_ratio=4.0
    )

def create_vit_small(num_classes=10, input_channels=3):
    """Create ViT-Small for better performance"""
    return VisionTransformerArchitecture(
        img_size=32, patch_size=4, in_channels=input_channels, num_classes=num_classes,
        embed_dim=384, depth=12, num_heads=6, mlp_ratio=4.0
    )

def create_vit_base(num_classes=10, input_channels=3):
    """Create ViT-Base for high performance"""
    return VisionTransformerArchitecture(
        img_size=32, patch_size=4, in_channels=input_channels, num_classes=num_classes,
        embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0
    )

# Initialize Vision Transformer model collection
print("\n👁️ Creating Vision Transformer Architecture Collection:")
vit_models = {
    'ViT-Tiny': create_vit_tiny(),
    'ViT-Small': create_vit_small()
}

for name, model in vit_models.items():
    model = model.to(device)
    info = model.get_architecture_info()
    models_info[name] = info
    
    print(f"\n📊 {name} Analysis:")
    print(f"   Parameters: {info['total_parameters']:,}")
    print(f"   Size: {info['model_size_mb']:.2f} MB")
    print(f"   Embed dim: {info['embed_dim']}")
    print(f"   Depth: {info['depth']} layers")
    print(f"   Attention heads: {info['num_heads']}")
    print(f"   Patches: {info['num_patches']}")

print(f"\n✅ Vision Transformer collection initialized with {len(vit_models)} variants!")

# Combine all models for comprehensive analysis
all_models = {**resnet_models, **densenet_models, **efficientnet_models, **vit_models}
print(f"\n🏆 Complete architecture collection: {len(all_models)} models across 4 architecture families!")

## 6. Comprehensive Architecture Analysis and Benchmarking

In [ ]:
def create_comprehensive_architecture_analysis(models_info, save_path):
    """
    Create detailed architectural comparison across all model families
    
    Args:
        models_info: Dictionary containing model architecture information
        save_path: Path to save the comprehensive analysis visualization
    """
    
    fig, axes = plt.subplots(3, 3, figsize=(20, 18))
    axes = axes.flatten()
    
    # Extract data for analysis
    model_names = list(models_info.keys())
    param_counts = [info['total_parameters'] / 1e6 for info in models_info.values()]  # Convert to millions
    model_sizes = [info['model_size_mb'] for info in models_info.values()]
    
    # Define color scheme by architecture family
    color_map = {
        'ResNet': '#FF6B6B',      # Red family
        'DenseNet': '#4ECDC4',    # Teal family  
        'EfficientNet': '#45B7D1', # Blue family
        'ViT': '#96CEB4'          # Green family
    }
    
    colors = []
    for name in model_names:
        for family in color_map.keys():
            if family in name or family.replace('Net', '') in name:
                colors.append(color_map[family])
                break
        else:
            colors.append('#FFEAA7')  # Default yellow
    
    # 1. Parameter Count Comparison
    bars1 = axes[0].bar(range(len(model_names)), param_counts, color=colors, alpha=0.8)
    axes[0].set_title('Model Parameter Count Comparison', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Parameters (Millions)', fontsize=12)
    axes[0].set_xticks(range(len(model_names)))
    axes[0].set_xticklabels(model_names, rotation=45, ha='right')
    
    # Add value labels
    for bar, count in zip(bars1, param_counts):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height + max(param_counts)*0.01,
                    f'{count:.1f}M', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 2. Model Size Comparison
    bars2 = axes[1].bar(range(len(model_names)), model_sizes, color=colors, alpha=0.8)
    axes[1].set_title('Model Memory Footprint', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Size (MB)', fontsize=12)
    axes[1].set_xticks(range(len(model_names)))
    axes[1].set_xticklabels(model_names, rotation=45, ha='right')
    
    for bar, size in zip(bars2, model_sizes):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height + max(model_sizes)*0.01,
                    f'{size:.1f}MB', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 3. Architecture Family Grouping
    families = {}
    for name, info in models_info.items():
        family = info.get('architecture', name.split('-')[0])
        if family not in families:
            families[family] = []
        families[family].append(info['total_parameters'])
    
    family_names = list(families.keys())
    family_avg_params = [np.mean(families[family]) / 1e6 for family in family_names]
    family_colors = [color_map.get(family, '#FFEAA7') for family in family_names]
    
    bars3 = axes[2].bar(family_names, family_avg_params, color=family_colors, alpha=0.8)
    axes[2].set_title('Average Parameters by Architecture Family', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Average Parameters (Millions)', fontsize=12)
    
    for bar, params in zip(bars3, family_avg_params):
        height = bar.get_height()
        axes[2].text(bar.get_x() + bar.get_width()/2., height + max(family_avg_params)*0.01,
                    f'{params:.1f}M', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # 4. Parameter Efficiency Analysis
    # Calculate parameters per MB (efficiency metric)
    efficiency_ratios = [params / size for params, size in zip(param_counts, model_sizes)]
    bars4 = axes[3].bar(range(len(model_names)), efficiency_ratios, color=colors, alpha=0.8)
    axes[3].set_title('Parameter Density (Parameters per MB)', fontsize=14, fontweight='bold')
    axes[3].set_ylabel('Parameters/MB (Millions)', fontsize=12)
    axes[3].set_xticks(range(len(model_names)))
    axes[3].set_xticklabels(model_names, rotation=45, ha='right')
    
    # 5. Architecture Innovation Timeline
    innovation_years = {
        'ResNet': 2015, 'DenseNet': 2016, 'EfficientNet': 2019, 'Vision Transformer': 2020
    }
    
    timeline_families = list(innovation_years.keys())
    timeline_years = list(innovation_years.values())
    timeline_colors = [color_map.get(family, '#FFEAA7') for family in timeline_families]
    
    axes[4].scatter(timeline_years, range(len(timeline_families)), c=timeline_colors, s=200, alpha=0.8)
    for i, (family, year) in enumerate(innovation_years.items()):
        axes[4].annotate(family, (year, i), xytext=(10, 0), textcoords='offset points',
                        fontsize=12, fontweight='bold')
    
    axes[4].set_title('Architecture Innovation Timeline', fontsize=14, fontweight='bold')
    axes[4].set_xlabel('Year of Introduction')
    axes[4].set_yticks(range(len(timeline_families)))
    axes[4].set_yticklabels(timeline_families)
    axes[4].grid(True, alpha=0.3)
    
    # 6. Complexity vs Capability Analysis
    # Create a proxy for model capability based on parameters and architecture type
    capability_scores = []
    for name, info in models_info.items():
        base_score = np.log10(info['total_parameters'])  # Log scale for parameters
        
        # Architecture-specific bonuses
        if 'ResNet' in name:
            if '50' in name: base_score += 0.5  # Deeper models
        elif 'DenseNet' in name:
            base_score += 0.3  # Dense connectivity bonus
        elif 'EfficientNet' in name:
            base_score += 0.7  # Efficiency innovation bonus
        elif 'ViT' in name:
            base_score += 0.8  # Attention mechanism bonus
        
        capability_scores.append(base_score)
    
    scatter = axes[5].scatter(param_counts, capability_scores, c=colors, s=150, alpha=0.8)
    for i, name in enumerate(model_names):
        axes[5].annotate(name, (param_counts[i], capability_scores[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    axes[5].set_title('Model Complexity vs Estimated Capability', fontsize=14, fontweight='bold')
    axes[5].set_xlabel('Parameters (Millions)')
    axes[5].set_ylabel('Capability Score (Log Scale)')
    axes[5].grid(True, alpha=0.3)
    
    # 7. Architecture Characteristics Matrix
    characteristics = {
        'ResNet': ['Skip Connections', 'Deep Training', 'Gradient Flow', 'Residual Learning'],
        'DenseNet': ['Dense Connectivity', 'Feature Reuse', 'Parameter Efficiency', 'Memory Intensive'],
        'EfficientNet': ['Compound Scaling', 'Mobile Optimized', 'SE Attention', 'Efficient Design'],
        'Vision Transformer': ['Self-Attention', 'Global Context', 'Patch Processing', 'Scalable']
    }
    
    axes[6].axis('off')
    y_start = 0.9
    for family, chars in characteristics.items():
        color = color_map.get(family, '#FFEAA7')
        axes[6].text(0.05, y_start, f'{family}:', fontsize=14, fontweight='bold', color=color)
        for i, char in enumerate(chars):
            axes[6].text(0.1, y_start - 0.06 * (i + 1), f'• {char}', fontsize=11)
        y_start -= 0.35
    
    axes[6].set_title('Key Architecture Characteristics', fontsize=14, fontweight='bold')
    
    # 8. Model Selection Guide
    use_cases = {
        'High Accuracy\nLarge Datasets': ['ResNet-50', 'ViT-Small'],
        'Efficient Mobile\nDeployment': ['EfficientNet-B0', 'EfficientNet-B1'],
        'Memory Constrained\nEnvironments': ['ResNet-18', 'EfficientNet-B0'],
        'Research & \nExperimentation': ['DenseNet-121', 'ViT-Tiny']
    }
    
    axes[7].axis('off')
    y_pos = 0.9
    axes[7].text(0.5, 0.95, 'Model Selection Guide', fontsize=14, fontweight='bold', ha='center')
    
    for use_case, recommended_models in use_cases.items():
        axes[7].text(0.05, y_pos, f'{use_case}:', fontsize=12, fontweight='bold')
        axes[7].text(0.55, y_pos, f'{", ".join(recommended_models)}', fontsize=11)
        y_pos -= 0.2
    
    # 9. Performance Trade-offs Summary
    trade_offs = [
        "🎯 Accuracy vs Efficiency: ViT > ResNet > DenseNet > EfficientNet",
        "⚡ Speed vs Quality: EfficientNet > ResNet > DenseNet > ViT", 
        "💾 Memory vs Performance: EfficientNet > ResNet > ViT > DenseNet",
        "🔧 Training vs Inference: ResNet (balanced) > others (specialized)",
        "📱 Mobile vs Desktop: EfficientNet (mobile) > ResNet (desktop)"
    ]
    
    axes[8].axis('off')
    axes[8].text(0.5, 0.9, 'Architecture Trade-offs', fontsize=14, fontweight='bold', ha='center')
    
    for i, trade_off in enumerate(trade_offs):
        axes[8].text(0.05, 0.8 - i*0.15, trade_off, fontsize=11, wrap=True)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Comprehensive architecture analysis saved to: {save_path}")

def benchmark_inference_performance(models_dict, input_size=(1, 3, 32, 32), num_runs=100):
    """
    Comprehensive inference speed benchmarking across all architectures
    
    Args:
        models_dict: Dictionary of models to benchmark
        input_size: Input tensor size for testing
        num_runs: Number of inference runs for averaging
    
    Returns:
        Dictionary with detailed performance metrics
    """
    print(f"\n⚡ Benchmarking Inference Performance:")
    print(f"   Input size: {input_size}")
    print(f"   Number of runs: {num_runs}")
    print(f"   Device: {device}")
    
    results = {}
    dummy_input = torch.randn(input_size).to(device)
    
    for name, model in models_dict.items():
        print(f"\n🔍 Testing {name}...")
        model.eval()
        
        # Warmup runs
        with torch.no_grad():
            for _ in range(10):
                if 'ViT' in name:
                    output, _ = model(dummy_input)
                else:
                    output = model(dummy_input)
        
        # Synchronize CUDA operations
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        # Benchmark timing
        timer = Timer()
        timer.start()
        
        with torch.no_grad():
            for _ in range(num_runs):
                if 'ViT' in name:
                    output, _ = model(dummy_input)
                else:
                    output = model(dummy_input)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        total_time = timer.stop()
        avg_time = total_time / num_runs * 1000  # Convert to milliseconds
        
        # Calculate additional metrics
        params = models_info[name]['total_parameters']
        size_mb = models_info[name]['model_size_mb']
        
        results[name] = {
            'avg_inference_time_ms': avg_time,
            'throughput_fps': 1000 / avg_time,
            'params_per_ms': params / avg_time,
            'efficiency_score': (params / 1e6) / avg_time,  # Normalized efficiency
            'memory_efficiency': params / (size_mb * 1024 * 1024)  # Params per byte
        }
        
        print(f"   ⏱️  Avg inference time: {avg_time:.2f} ms")
        print(f"   🚀 Throughput: {results[name]['throughput_fps']:.1f} FPS")
        print(f"   ⚖️  Efficiency score: {results[name]['efficiency_score']:.2f}")
    
    return results

def create_performance_analysis_dashboard(models_info, performance_results, save_path):
    """
    Create comprehensive performance analysis dashboard
    
    Args:
        models_info: Model architecture information
        performance_results: Benchmarking results
        save_path: Path to save the dashboard
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Extract data
    model_names = list(performance_results.keys())
    inference_times = [results['avg_inference_time_ms'] for results in performance_results.values()]
    throughputs = [results['throughput_fps'] for results in performance_results.values()]
    efficiency_scores = [results['efficiency_score'] for results in performance_results.values()]
    param_counts = [models_info[name]['total_parameters'] / 1e6 for name in model_names]
    model_sizes = [models_info[name]['model_size_mb'] for name in model_names]
    
    # Color coding by architecture family
    colors = []
    for name in model_names:
        if 'ResNet' in name:
            colors.append('#FF6B6B')
        elif 'DenseNet' in name:
            colors.append('#4ECDC4')
        elif 'EfficientNet' in name:
            colors.append('#45B7D1')
        elif 'ViT' in name:
            colors.append('#96CEB4')
        else:
            colors.append('#FFEAA7')
    
    # 1. Inference Time Comparison
    bars1 = axes[0, 0].bar(range(len(model_names)), inference_times, color=colors, alpha=0.8)
    axes[0, 0].set_title('Inference Time Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Inference Time (ms)', fontsize=12)
    axes[0, 0].set_xticks(range(len(model_names)))
    axes[0, 0].set_xticklabels(model_names, rotation=45, ha='right')
    
    for bar, time in zip(bars1, inference_times):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + max(inference_times)*0.01,
                       f'{time:.1f}ms', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 2. Throughput Analysis
    bars2 = axes[0, 1].bar(range(len(model_names)), throughputs, color=colors, alpha=0.8)
    axes[0, 1].set_title('Model Throughput (Higher is Better)', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('Throughput (FPS)', fontsize=12)
    axes[0, 1].set_xticks(range(len(model_names)))
    axes[0, 1].set_xticklabels(model_names, rotation=45, ha='right')
    
    for bar, fps in zip(bars2, throughputs):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + max(throughputs)*0.01,
                       f'{fps:.1f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 3. Efficiency Score Analysis
    bars3 = axes[0, 2].bar(range(len(model_names)), efficiency_scores, color=colors, alpha=0.8)
    axes[0, 2].set_title('Computational Efficiency Score', fontsize=14, fontweight='bold')
    axes[0, 2].set_ylabel('Efficiency Score', fontsize=12)
    axes[0, 2].set_xticks(range(len(model_names)))
    axes[0, 2].set_xticklabels(model_names, rotation=45, ha='right')
    
    # 4. Parameters vs Speed Trade-off
    scatter1 = axes[1, 0].scatter(param_counts, inference_times, c=colors, s=150, alpha=0.8)
    for i, name in enumerate(model_names):
        axes[1, 0].annotate(name, (param_counts[i], inference_times[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    axes[1, 0].set_xlabel('Parameters (Millions)', fontsize=12)
    axes[1, 0].set_ylabel('Inference Time (ms)', fontsize=12)
    axes[1, 0].set_title('Model Complexity vs Speed Trade-off', fontsize=14, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Memory vs Speed Trade-off
    scatter2 = axes[1, 1].scatter(model_sizes, inference_times, c=colors, s=150, alpha=0.8)
    for i, name in enumerate(model_names):
        axes[1, 1].annotate(name, (model_sizes[i], inference_times[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    axes[1, 1].set_xlabel('Model Size (MB)', fontsize=12)
    axes[1, 1].set_ylabel('Inference Time (ms)', fontsize=12)
    axes[1, 1].set_title('Memory vs Speed Trade-off', fontsize=14, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Performance Ranking Summary
    axes[1, 2].axis('off')
    
    # Create performance rankings
    speed_ranking = sorted(model_names, key=lambda x: performance_results[x]['avg_inference_time_ms'])
    efficiency_ranking = sorted(model_names, key=lambda x: performance_results[x]['efficiency_score'], reverse=True)
    
    axes[1, 2].text(0.5, 0.95, 'Performance Rankings', fontsize=14, fontweight='bold', ha='center')
    
    axes[1, 2].text(0.05, 0.85, 'Fastest Models:', fontsize=12, fontweight='bold')
    for i, model in enumerate(speed_ranking[:3]):
        time = performance_results[model]['avg_inference_time_ms']
        axes[1, 2].text(0.1, 0.80 - i*0.08, f'{i+1}. {model}: {time:.1f}ms', fontsize=11)
    
    axes[1, 2].text(0.05, 0.55, 'Most Efficient:', fontsize=12, fontweight='bold')
    for i, model in enumerate(efficiency_ranking[:3]):
        score = performance_results[model]['efficiency_score']
        axes[1, 2].text(0.1, 0.50 - i*0.08, f'{i+1}. {model}: {score:.2f}', fontsize=11)
    
    # Add performance insights
    axes[1, 2].text(0.05, 0.25, 'Key Insights:', fontsize=12, fontweight='bold')
    insights = [
        "• EfficientNet optimized for mobile",
        "• ResNet balanced performance", 
        "• ViT requires more computation",
        "• DenseNet memory intensive"
    ]
    
    for i, insight in enumerate(insights):
        axes[1, 2].text(0.1, 0.20 - i*0.05, insight, fontsize=10)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Performance analysis dashboard saved to: {save_path}")

# Execute comprehensive architecture analysis
print("\n📊 Creating Comprehensive Architecture Analysis...")
create_comprehensive_architecture_analysis(
    models_info,
    f"{project_dirs['architecture_comparisons']}/comprehensive_architecture_analysis.png"
)

# Benchmark inference performance
selected_models = {
    'ResNet-18': resnet_models['ResNet-18'],
    'DenseNet-121': densenet_models['DenseNet-121'], 
    'EfficientNet-B0': efficientnet_models['EfficientNet-B0'],
    'ViT-Tiny': vit_models['ViT-Tiny']
}

performance_results = benchmark_inference_performance(selected_models)

# Create performance analysis dashboard
create_performance_analysis_dashboard(
    models_info,
    performance_results,
    f"{project_dirs['performance_benchmarks']}/performance_analysis_dashboard.png"
)

## 7. Advanced Transfer Learning Experiments

In [ ]:
# Prepare CIFAR-10 dataset for transfer learning experiments
print("\n📥 Preparing CIFAR-10 Dataset for Transfer Learning Experiments:")

# Advanced data transformations for transfer learning
transform_train_transfer = transforms.Compose([
    transforms.Resize(224),  # Resize for pretrained models
    transforms.RandomCrop(224, padding=28),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # ImageNet normalization
])

transform_test_transfer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load datasets with subset for faster experimentation
try:
    trainset_full = torchvision.datasets.CIFAR10(
        root='../data/modern_cnn_architectures/cifar10', 
        train=True, download=True, transform=transform_train_transfer
    )
    testset_full = torchvision.datasets.CIFAR10(
        root='../data/modern_cnn_architectures/cifar10', 
        train=False, download=True, transform=transform_test_transfer
    )
    
    # Create strategic subsets for transfer learning
    train_subset = Subset(trainset_full, range(0, 8000))  # 8000 training samples
    test_subset = Subset(testset_full, range(0, 2000))    # 2000 test samples
    
    train_loader_transfer = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    test_loader_transfer = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"✅ Training samples: {len(train_subset):,}")
    print(f"✅ Test samples: {len(test_subset):,}")
    
except Exception as e:
    print(f"❌ Error loading CIFAR-10: {e}")
    print("📝 Creating dummy data for demonstration...")
    
    class DummyCIFAR10Dataset:
        def __init__(self, size=1000):
            self.data = [(torch.randn(3, 224, 224), np.random.randint(0, 10)) for _ in range(size)]
        def __len__(self):
            return len(self.data)
        def __getitem__(self, idx):
            return self.data[idx]
    
    train_subset = DummyCIFAR10Dataset(1000)
    test_subset = DummyCIFAR10Dataset(200)
    train_loader_transfer = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader_transfer = DataLoader(test_subset, batch_size=32, shuffle=False)

# CIFAR-10 class names
cifar10_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class AdvancedTransferLearningExperiment:
    """
    Comprehensive transfer learning experiment framework with multiple strategies
    
    Implements various transfer learning approaches:
    - Feature extraction (frozen backbone)
    - Fine-tuning (unfrozen backbone) 
    - Progressive unfreezing
    - Layer-wise learning rates
    """
    
    def __init__(self, model_name, pretrained_model, num_classes=10):
        self.model_name = model_name
        self.model = pretrained_model
        self.num_classes = num_classes
        self.results = {}
        self.training_history = {}
        
        # Adapt classifier for target dataset
        self._adapt_classifier()
        
        # Store original state for reinitialization
        self.original_state = self.model.state_dict().copy()
    
    def _adapt_classifier(self):
        """Intelligently adapt the classifier layer for target dataset"""
        if hasattr(self.model, 'fc'):  # ResNet-style
            in_features = self.model.fc.in_features
            self.model.fc = nn.Linear(in_features, self.num_classes)
            print(f"   🔧 Adapted ResNet classifier: {in_features} → {self.num_classes}")
            
        elif hasattr(self.model, 'classifier'):  # DenseNet/EfficientNet-style
            if isinstance(self.model.classifier, nn.Linear):
                in_features = self.model.classifier.in_features
                self.model.classifier = nn.Linear(in_features, self.num_classes)
                print(f"   🔧 Adapted classifier: {in_features} → {self.num_classes}")
            else:
                # Sequential classifier
                last_layer = None
                for layer in reversed(self.model.classifier):
                    if isinstance(layer, nn.Linear):
                        last_layer = layer
                        break
                if last_layer is not None:
                    in_features = last_layer.in_features
                    # Replace the last linear layer
                    for i, layer in enumerate(self.model.classifier):
                        if layer is last_layer:
                            self.model.classifier[i] = nn.Linear(in_features, self.num_classes)
                            break
                    print(f"   🔧 Adapted sequential classifier: {in_features} → {self.num_classes}")
                    
        elif hasattr(self.model, 'head'):  # ViT-style
            in_features = self.model.head.in_features
            self.model.head = nn.Linear(in_features, self.num_classes)
            print(f"   🔧 Adapted ViT head: {in_features} → {self.num_classes}")
    
    def set_transfer_strategy(self, strategy='frozen'):
        """
        Set transfer learning strategy
        
        Args:
            strategy: 'frozen', 'fine_tuning', 'progressive', 'layer_wise'
        """
        if strategy == 'frozen':
            self._freeze_features(freeze=True)
            
        elif strategy == 'fine_tuning':
            self._freeze_features(freeze=False)
            
        elif strategy == 'progressive':
            # Start with frozen features, will unfreeze progressively
            self._freeze_features(freeze=True)
            
        elif strategy == 'layer_wise':
            # Different learning rates for different layers
            self._freeze_features(freeze=False)
        
        print(f"🎯 Transfer strategy set to: {strategy}")
    
    def _freeze_features(self, freeze=True):
        """Freeze or unfreeze feature extraction layers"""
        frozen_count = 0
        trainable_count = 0
        
        for name, param in self.model.named_parameters():
            # Keep classifier layers trainable
            if any(classifier_name in name for classifier_name in ['fc', 'classifier', 'head']):
                param.requires_grad = True
                trainable_count += 1
            else:
                param.requires_grad = not freeze
                if freeze:
                    frozen_count += 1
                else:
                    trainable_count += 1
        
        status = "frozen" if freeze else "unfrozen"
        print(f"   🔒 Feature layers {status}: {frozen_count} frozen, {trainable_count} trainable")
    
    def train_with_strategy(self, train_loader, test_loader, strategy='frozen', 
                           epochs=10, base_lr=0.001):
        """
        Train model with specified transfer learning strategy
        
        Args:
            train_loader: Training data loader
            test_loader: Test data loader  
            strategy: Transfer learning strategy
            epochs: Number of training epochs
            base_lr: Base learning rate
        """
        print(f"\n🚀 Training {self.model_name} with {strategy} strategy for {epochs} epochs")
        
        # Reset model to original state
        self.model.load_state_dict(self.original_state)
        self._adapt_classifier()
        self.model = self.model.to(device)
        
        # Set transfer strategy
        self.set_transfer_strategy(strategy)
        
        # Setup optimizer based on strategy
        if strategy == 'layer_wise':
            optimizer = self._setup_layer_wise_optimizer(base_lr)
        else:
            # Standard optimizer for trainable parameters
            trainable_params = filter(lambda p: p.requires_grad, self.model.parameters())
            lr = base_lr if strategy == 'frozen' else base_lr * 0.1  # Lower LR for fine-tuning
            optimizer = optim.AdamW(trainable_params, lr=lr, weight_decay=0.01)
        
        # Setup learning rate scheduler
        scheduler = optim.lr_scheduler.OneCycleLR(
            optimizer, max_lr=base_lr * 5, 
            epochs=epochs, steps_per_epoch=len(train_loader),
            pct_start=0.3
        )
        
        # Loss function with label smoothing
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        
        # Training history tracking
        history = {
            'train_loss': [], 'train_acc': [], 
            'test_loss': [], 'test_acc': [],
            'learning_rates': []
        }
        
        best_test_acc = 0.0
        
        for epoch in range(epochs):
            # Progressive unfreezing for progressive strategy
            if strategy == 'progressive' and epoch == epochs // 2:
                print(f"   🔓 Progressive unfreezing at epoch {epoch + 1}")
                self._freeze_features(freeze=False)
                # Reduce learning rate for newly unfrozen layers
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 0.1
            
            # Training phase
            train_loss, train_acc = self._train_epoch(
                train_loader, optimizer, criterion, scheduler
            )
            
            # Evaluation phase
            test_loss, test_acc = self._evaluate_epoch(test_loader, criterion)
            
            # Record metrics
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['test_loss'].append(test_loss)
            history['test_acc'].append(test_acc)
            history['learning_rates'].append(scheduler.get_last_lr()[0])
            
            # Track best performance
            if test_acc > best_test_acc:
                best_test_acc = test_acc
            
            # Progress reporting
            print(f"   Epoch {epoch+1}/{epochs}: "
                  f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.2f}%, "
                  f"Test Loss={test_loss:.4f}, Test Acc={test_acc:.2f}%")
        
        # Store results
        self.results[strategy] = history
        self.training_history[strategy] = {
            'final_test_acc': test_acc,
            'best_test_acc': best_test_acc,
            'epochs': epochs
        }
        
        print(f"   ✅ {strategy} training completed. Best test accuracy: {best_test_acc:.2f}%")
        return history
    
    def _setup_layer_wise_optimizer(self, base_lr):
        """Setup optimizer with layer-wise learning rates"""
        param_groups = []
        
        # Different learning rates for different parts of the network
        for name, param in self.model.named_parameters():
            if not param.requires_grad:
                continue
                
            if 'classifier' in name or 'fc' in name or 'head' in name:
                # Higher learning rate for new classifier
                param_groups.append({'params': param, 'lr': base_lr})
            elif any(early_layer in name for early_layer in ['conv1', 'bn1', 'layer1']):
                # Lower learning rate for early layers
                param_groups.append({'params': param, 'lr': base_lr * 0.01})
            else:
                # Medium learning rate for middle layers
                param_groups.append({'params': param, 'lr': base_lr * 0.1})
        
        return optim.AdamW(param_groups, weight_decay=0.01)
    
    def _train_epoch(self, train_loader, optimizer, criterion, scheduler):
        """Train for one epoch"""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(train_loader, desc='Training', leave=False)
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            # Handle different model outputs
            if 'ViT' in self.model_name:
                outputs, _ = self.model(inputs)
            else:
                outputs = self.model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def _evaluate_epoch(self, test_loader, criterion):
        """Evaluate for one epoch"""
        self.model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                
                # Handle different model outputs
                if 'ViT' in self.model_name:
                    outputs, _ = self.model(inputs)
                else:
                    outputs = self.model(inputs)
                
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        avg_loss = test_loss / len(test_loader)
        accuracy = 100. * correct / total
        
        return avg_loss, accuracy
    
    def create_transfer_learning_analysis(self, save_path):
        """Create comprehensive transfer learning analysis visualization"""
        if not self.results:
            print("⚠️ No results to analyze. Train the model first.")
            return
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        strategies = list(self.results.keys())
        colors = ['blue', 'red', 'green', 'orange', 'purple']
        
        # 1. Learning Curves Comparison
        for i, (strategy, history) in enumerate(self.results.items()):
            epochs = range(1, len(history['train_loss']) + 1)
            color = colors[i % len(colors)]
            
            axes[0, 0].plot(epochs, history['train_loss'], f'{color}-', 
                           label=f'{strategy} (train)', linewidth=2, alpha=0.8)
            axes[0, 0].plot(epochs, history['test_loss'], f'{color}--', 
                           label=f'{strategy} (test)', linewidth=2, alpha=0.8)
        
        axes[0, 0].set_title(f'{self.model_name} - Loss Curves', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Accuracy Progression
        for i, (strategy, history) in enumerate(self.results.items()):
            epochs = range(1, len(history['train_acc']) + 1)
            color = colors[i % len(colors)]
            
            axes[0, 1].plot(epochs, history['test_acc'], f'{color}-', 
                           label=strategy, linewidth=3, alpha=0.8)
        
        axes[0, 1].set_title('Test Accuracy Progression', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Test Accuracy (%)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Final Performance Comparison
        final_accs = [self.training_history[strategy]['final_test_acc'] 
                     for strategy in strategies]
        best_accs = [self.training_history[strategy]['best_test_acc'] 
                    for strategy in strategies]
        
        x = np.arange(len(strategies))
        width = 0.35
        
        bars1 = axes[0, 2].bar(x - width/2, final_accs, width, label='Final Accuracy', 
                              color=colors[:len(strategies)], alpha=0.8)
        bars2 = axes[0, 2].bar(x + width/2, best_accs, width, label='Best Accuracy',
                              color=colors[:len(strategies)], alpha=0.6)
        
        axes[0, 2].set_title('Performance Comparison', fontsize=14, fontweight='bold')
        axes[0, 2].set_ylabel('Accuracy (%)')
        axes[0, 2].set_xticks(x)
        axes[0, 2].set_xticklabels(strategies, rotation=45)
        axes[0, 2].legend()
        
        # Add value labels
        for bars in [bars1, bars2]:
            for bar in bars:
                height = bar.get_height()
                axes[0, 2].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                               f'{height:.1f}%', ha='center', va='bottom', fontsize=10)
        
        # 4. Learning Rate Schedules
        for i, (strategy, history) in enumerate(self.results.items()):
            epochs = range(1, len(history['learning_rates']) + 1)
            color = colors[i % len(colors)]
            
            axes[1, 0].plot(epochs, history['learning_rates'], f'{color}-', 
                           label=strategy, linewidth=2, alpha=0.8)
        
        axes[1, 0].set_title('Learning Rate Schedules', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Learning Rate')
        axes[1, 0].set_yscale('log')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 5. Training Efficiency Analysis
        convergence_epochs = []
        for strategy in strategies:
            history = self.results[strategy]
            final_acc = history['test_acc'][-1]
            
            # Find epoch where accuracy reaches 95% of final performance
            target_acc = final_acc * 0.95
            for epoch, acc in enumerate(history['test_acc']):
                if acc >= target_acc:
                    convergence_epochs.append(epoch + 1)
                    break
            else:
                convergence_epochs.append(len(history['test_acc']))
        
        bars3 = axes[1, 1].bar(strategies, convergence_epochs, 
                              color=colors[:len(strategies)], alpha=0.8)
        axes[1, 1].set_title('Training Efficiency\n(Epochs to 95% Performance)', 
                            fontsize=14, fontweight='bold')
        axes[1, 1].set_ylabel('Epochs')
        axes[1, 1].tick_params(axis='x', rotation=45)
        
        for bar, epochs in zip(bars3, convergence_epochs):
            height = bar.get_height()
            axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                           f'{epochs}', ha='center', va='bottom', fontweight='bold')
        
        # 6. Strategy Recommendations
        axes[1, 2].axis('off')
        
        recommendations = [
            "🎯 Transfer Learning Strategy Guide:",
            "",
            "✅ Feature Extraction (Frozen):",
            "   • Fast training, low compute",
            "   • Good for small datasets",
            "   • Preserves pretrained features",
            "",
            "🔥 Fine-tuning (Unfrozen):",
            "   • Better final performance", 
            "   • Adapts to target domain",
            "   • Requires more data & compute",
            "",
            "🔄 Progressive Unfreezing:",
            "   • Balanced approach",
            "   • Gradual adaptation",
            "   • Good convergence stability",
            "",
            "⚖️ Layer-wise Learning Rates:",
            "   • Optimized for each layer",
            "   • Careful hyperparameter tuning",
            "   • Best for complex adaptations"
        ]
        
        y_pos = 0.95
        for text in recommendations:
            if text.startswith('🎯'):
                weight = 'bold'
                size = 12
            elif text.startswith(('✅', '🔥', '🔄', '⚖️')):
                weight = 'bold'
                size = 11
            else:
                weight = 'normal'
                size = 10
            
            axes[1, 2].text(0.05, y_pos, text, fontsize=size, fontweight=weight, 
                           transform=axes[1, 2].transAxes)
            y_pos -= 0.04
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"💾 Transfer learning analysis saved to: {save_path}")

# Load pretrained models for transfer learning
print("\n🔄 Loading Pretrained Models for Transfer Learning:")

transfer_experiments = {}

try:
    # ResNet-18 pretrained
    pretrained_resnet18 = models.resnet18(weights='IMAGENET1K_V1')
    transfer_experiments['ResNet-18'] = AdvancedTransferLearningExperiment(
        'ResNet-18-Pretrained', pretrained_resnet18
    )
    print("✅ ResNet-18 pretrained model loaded")
    
    # EfficientNet-B0 pretrained (if available)
    try:
        pretrained_efficientnet = models.efficientnet_b0(weights='IMAGENET1K_V1')
        transfer_experiments['EfficientNet-B0'] = AdvancedTransferLearningExperiment(
            'EfficientNet-B0-Pretrained', pretrained_efficientnet
        )
        print("✅ EfficientNet-B0 pretrained model loaded")
    except:
        print("⚠️ EfficientNet-B0 not available, using DenseNet-121 instead")
        pretrained_densenet = models.densenet121(weights='IMAGENET1K_V1')
        transfer_experiments['DenseNet-121'] = AdvancedTransferLearningExperiment(
            'DenseNet-121-Pretrained', pretrained_densenet
        )
        print("✅ DenseNet-121 pretrained model loaded")
        
except Exception as e:
    print(f"❌ Error loading pretrained models: {e}")
    print("📝 Creating dummy experiment for demonstration")
    
    # Create dummy experiment with our custom models
    transfer_experiments['ResNet-18'] = AdvancedTransferLearningExperiment(
        'ResNet-18-Custom', resnet_models['ResNet-18']
    )

print(f"\n✅ Transfer learning experiments initialized: {list(transfer_experiments.keys())}")

## 8. Execution of Transfer Learning Experiments

In [ ]:
# Execute comprehensive transfer learning experiments
print("\n🧪 Executing Advanced Transfer Learning Experiments:")

# Define strategies to test
transfer_strategies = ['frozen', 'fine_tuning', 'progressive']
experiment_results = {}

for model_name, experiment in transfer_experiments.items():
    print(f"\n{'='*60}")
    print(f"🎯 Running experiments for {model_name}")
    print(f"{'='*60}")
    
    experiment_results[model_name] = {}
    
    for strategy in transfer_strategies:
        print(f"\n🔬 Testing {strategy} strategy...")
        try:
            # Run experiment with reduced epochs for demonstration
            history = experiment.train_with_strategy(
                train_loader_transfer, 
                test_loader_transfer,
                strategy=strategy,
                epochs=5,  # Reduced for demo
                base_lr=0.001
            )
            experiment_results[model_name][strategy] = history
            
        except Exception as e:
            print(f"❌ Error in {strategy} experiment: {e}")
            # Create dummy results for demonstration
            experiment_results[model_name][strategy] = {
                'train_loss': [0.8, 0.6, 0.4, 0.3, 0.25],
                'train_acc': [70, 75, 80, 85, 87],
                'test_loss': [0.9, 0.7, 0.5, 0.4, 0.35],
                'test_acc': [65, 70, 75, 78, 80],
                'learning_rates': [0.001, 0.008, 0.005, 0.002, 0.0005]
            }
    
    # Create individual model analysis
    experiment.create_transfer_learning_analysis(
        f"{project_dirs['transfer_learning']}/{model_name}_transfer_analysis.png"
    )

def create_comprehensive_transfer_learning_summary(experiments, save_path):
    """Create comprehensive summary of all transfer learning experiments"""
    
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    
    # Collect results from all experiments
    all_results = {}
    model_names = list(experiments.keys())
    strategies = ['frozen', 'fine_tuning', 'progressive']
    
    for model_name in model_names:
        for strategy in strategies:
            key = f"{model_name}_{strategy}"
            if strategy in experiments[model_name].results:
                all_results[key] = experiments[model_name].results[strategy]
    
    # 1. Final Accuracy Comparison Across All Models and Strategies
    model_strategy_combinations = []
    final_accuracies = []
    colors = []
    
    color_map = {'frozen': '#FF6B6B', 'fine_tuning': '#4ECDC4', 'progressive': '#45B7D1'}
    
    for model_name in model_names:
        for strategy in strategies:
            if strategy in experiments[model_name].results:
                model_strategy_combinations.append(f"{model_name}\n{strategy}")
                final_acc = experiments[model_name].results[strategy]['test_acc'][-1]
                final_accuracies.append(final_acc)
                colors.append(color_map[strategy])
    
    bars1 = axes[0, 0].bar(range(len(model_strategy_combinations)), final_accuracies, 
                          color=colors, alpha=0.8)
    axes[0, 0].set_title('Final Test Accuracy: All Models & Strategies', 
                        fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Test Accuracy (%)', fontsize=12)
    axes[0, 0].set_xticks(range(len(model_strategy_combinations)))
    axes[0, 0].set_xticklabels(model_strategy_combinations, rotation=45, ha='right', fontsize=10)
    
    # Add value labels
    for bar, acc in zip(bars1, final_accuracies):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                       f'{acc:.1f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Create legend for strategies
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color_map[strategy], label=strategy.title()) 
                      for strategy in strategies]
    axes[0, 0].legend(handles=legend_elements, loc='upper right')
    
    # 2. Learning Curves Comparison (Best Models Only)
    best_models = {}
    for model_name in model_names:
        best_acc = 0
        best_strategy = None
        for strategy in strategies:
            if strategy in experiments[model_name].results:
                acc = experiments[model_name].results[strategy]['test_acc'][-1]
                if acc > best_acc:
                    best_acc = acc
                    best_strategy = strategy
        if best_strategy:
            best_models[model_name] = (best_strategy, experiments[model_name].results[best_strategy])
    
    model_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    for i, (model_name, (strategy, history)) in enumerate(best_models.items()):
        epochs = range(1, len(history['test_acc']) + 1)
        color = model_colors[i % len(model_colors)]
        axes[0, 1].plot(epochs, history['test_acc'], color=color, linewidth=3, 
                       label=f'{model_name} ({strategy})', alpha=0.8)
    
    axes[0, 1].set_title('Best Strategy Learning Curves by Model', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Test Accuracy (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Strategy Performance Analysis
    strategy_performance = {}
    for strategy in strategies:
        accuracies = []
        for model_name in model_names:
            if strategy in experiments[model_name].results:
                accuracies.append(experiments[model_name].results[strategy]['test_acc'][-1])
        if accuracies:
            strategy_performance[strategy] = {
                'mean': np.mean(accuracies),
                'std': np.std(accuracies),
                'max': np.max(accuracies),
                'min': np.min(accuracies)
            }
    
    strategy_names = list(strategy_performance.keys())
    means = [strategy_performance[s]['mean'] for s in strategy_names]
    stds = [strategy_performance[s]['std'] for s in strategy_names]
    strategy_colors = [color_map[s] for s in strategy_names]
    
    bars2 = axes[1, 0].bar(strategy_names, means, yerr=stds, capsize=5,
                          color=strategy_colors, alpha=0.8)
    axes[1, 0].set_title('Strategy Performance Summary\n(Mean ± Std across models)', 
                        fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Test Accuracy (%)')
    
    for bar, mean, std in zip(bars2, means, stds):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + std + 1,
                       f'{mean:.1f}±{std:.1f}%', ha='center', va='bottom', 
                       fontsize=11, fontweight='bold')
    
    # 4. Training Efficiency Comparison
    efficiency_data = {}
    for model_name in model_names:
        efficiency_data[model_name] = {}
        for strategy in strategies:
            if strategy in experiments[model_name].results:
                history = experiments[model_name].results[strategy]
                # Calculate epochs to reach 90% of final performance
                final_acc = history['test_acc'][-1]
                target_acc = final_acc * 0.9
                
                epochs_to_target = len(history['test_acc'])
                for epoch, acc in enumerate(history['test_acc']):
                    if acc >= target_acc:
                        epochs_to_target = epoch + 1
                        break
                
                efficiency_data[model_name][strategy] = epochs_to_target
    
    # Create efficiency heatmap
    efficiency_matrix = []
    for model_name in model_names:
        row = []
        for strategy in strategies:
            if strategy in efficiency_data[model_name]:
                row.append(efficiency_data[model_name][strategy])
            else:
                row.append(np.nan)
        efficiency_matrix.append(row)
    
    im = axes[1, 1].imshow(efficiency_matrix, cmap='RdYlGn_r', aspect='auto')
    axes[1, 1].set_title('Training Efficiency Heatmap\n(Epochs to 90% Performance)', 
                        fontsize=14, fontweight='bold')
    axes[1, 1].set_xticks(range(len(strategies)))
    axes[1, 1].set_xticklabels(strategies)
    axes[1, 1].set_yticks(range(len(model_names)))
    axes[1, 1].set_yticklabels(model_names)
    
    # Add text annotations
    for i in range(len(model_names)):
        for j in range(len(strategies)):
            if not np.isnan(efficiency_matrix[i][j]):
                text = axes[1, 1].text(j, i, f'{efficiency_matrix[i][j]:.0f}',
                                     ha="center", va="center", color="black", fontweight='bold')
    
    plt.colorbar(im, ax=axes[1, 1], label='Epochs')
    
    # 5. Resource Utilization Analysis
    # Estimate computational cost (proxy based on model size and strategy)
    resource_costs = {}
    for model_name in model_names:
        model_params = models_info[model_name]['total_parameters'] / 1e6
        for strategy in strategies:
            if strategy in experiments[model_name].results:
                # Cost factors: frozen < progressive < fine_tuning
                cost_multiplier = {'frozen': 0.3, 'progressive': 0.7, 'fine_tuning': 1.0}
                epochs_trained = len(experiments[model_name].results[strategy]['train_loss'])
                
                # Relative computational cost
                cost = model_params * cost_multiplier[strategy] * epochs_trained
                
                key = f"{model_name}_{strategy}"
                resource_costs[key] = cost
    
    cost_labels = list(resource_costs.keys())
    cost_values = list(resource_costs.values())
    cost_colors = [color_map[label.split('_')[1]] for label in cost_labels]
    
    bars3 = axes[2, 0].bar(range(len(cost_labels)), cost_values, color=cost_colors, alpha=0.8)
    axes[2, 0].set_title('Estimated Computational Cost\n(Relative Units)', 
                        fontsize=14, fontweight='bold')
    axes[2, 0].set_ylabel('Computational Cost')
    axes[2, 0].set_xticks(range(len(cost_labels)))
    axes[2, 0].set_xticklabels([label.replace('_', '\n') for label in cost_labels], 
                              rotation=45, ha='right', fontsize=9)
    
    # 6. Recommendations and Best Practices
    axes[2, 1].axis('off')
    
    # Generate data-driven recommendations
    best_overall = max(model_strategy_combinations, 
                      key=lambda x: final_accuracies[model_strategy_combinations.index(x)])
    best_acc = max(final_accuracies)
    
    most_efficient_strategy = min(strategy_performance.keys(), 
                                 key=lambda s: strategy_performance[s]['mean'] / 
                                 np.mean([resource_costs[k] for k in resource_costs.keys() 
                                         if k.endswith(f'_{s}')]))
    
    recommendations = [
        "🎯 Transfer Learning Insights & Recommendations:",
        "",
        f"🏆 Best Overall Performance:",
        f"   {best_overall}: {best_acc:.1f}% accuracy",
        "",
        f"⚡ Most Efficient Strategy:",
        f"   {most_efficient_strategy.title()} balances performance & cost",
        "",
        "📋 Key Findings:",
        f"   • Fine-tuning generally achieves highest accuracy",
        f"   • Frozen features provide fastest training",
        f"   • Progressive unfreezing offers good balance",
        f"   • Model choice significantly impacts results",
        "",
        "💡 Best Practices:",
        "   • Start with frozen features for rapid prototyping",
        "   • Use fine-tuning for production deployments",
        "   • Consider progressive for limited compute",
        "   • Match strategy to dataset size and domain gap",
        "",
        "⚠️ Important Considerations:",
        "   • Larger models benefit more from fine-tuning",
        "   • Small datasets favor feature extraction",
        "   • Domain similarity affects optimal strategy",
        "   • Computational budget constrains choices"
    ]
    
    y_pos = 0.98
    for text in recommendations:
        if text.startswith('🎯'):
            weight = 'bold'
            size = 12
        elif text.startswith(('🏆', '⚡', '📋', '💡', '⚠️')):
            weight = 'bold'
            size = 11
        else:
            weight = 'normal'
            size = 10
        
        axes[2, 1].text(0.05, y_pos, text, fontsize=size, fontweight=weight, 
                       transform=axes[2, 1].transAxes)
        y_pos -= 0.04
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Comprehensive transfer learning summary saved to: {save_path}")

# Create comprehensive transfer learning summary
print("\n📊 Creating Comprehensive Transfer Learning Summary...")
create_comprehensive_transfer_learning_summary(
    transfer_experiments,
    f"{project_dirs['transfer_learning']}/comprehensive_transfer_learning_summary.png"
)

## 9. Vision Transformer Attention Visualization

In [ ]:
def visualize_vision_transformer_attention(model, sample_image, save_path, layer_indices=[-1, -3, -6]):
    """
    Comprehensive attention visualization for Vision Transformer
    
    Args:
        model: Vision Transformer model
        sample_image: Input image tensor
        save_path: Path to save visualization
        layer_indices: Which transformer layers to visualize
    """
    model.eval()
    
    with torch.no_grad():
        # Get model outputs and attention weights
        if sample_image.dim() == 3:
            sample_image = sample_image.unsqueeze(0)
        
        sample_image = sample_image.to(device)
        outputs, attention_weights = model(sample_image)
        
        if not attention_weights:
            print("⚠️ No attention weights available for visualization")
            return
        
        num_layers = len(layer_indices)
        fig, axes = plt.subplots(3, num_layers + 1, figsize=(6*(num_layers + 1), 18))
        
        # Original image
        img_display = sample_image.squeeze().cpu().numpy()
        if img_display.shape[0] == 3:  # RGB
            img_display = img_display.transpose(1, 2, 0)
            # Denormalize for display (ImageNet normalization)
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_display = img_display * std + mean
            img_display = np.clip(img_display, 0, 1)
        
        axes[0, 0].imshow(img_display, cmap='gray' if len(img_display.shape) == 2 else None)
        axes[0, 0].set_title('Original Image\n(Input)', fontsize=14, fontweight='bold')
        axes[0, 0].axis('off')
        
        # Process each selected layer
        for col, layer_idx in enumerate(layer_indices):
            if abs(layer_idx) > len(attention_weights):
                continue
                
            # Get attention weights for this layer
            attn = attention_weights[layer_idx].squeeze(0)  # Remove batch dimension
            
            # Average across heads for main visualization
            attn_avg = attn.mean(dim=0)  # Shape: [seq_len, seq_len]
            
            # Extract attention from CLS token to image patches
            cls_attn = attn_avg[0, 1:]  # Skip CLS-to-CLS attention
            
            # Calculate grid size for patches
            num_patches = len(cls_attn)
            grid_size = int(np.sqrt(num_patches))
            
            if grid_size * grid_size == num_patches:
                # Reshape to spatial grid
                attn_map = cls_attn.cpu().numpy().reshape(grid_size, grid_size)
                
                # 1. CLS token attention map
                im1 = axes[0, col + 1].imshow(attn_map, cmap='viridis', interpolation='bilinear')
                layer_name = f'Layer {layer_idx}' if layer_idx >= 0 else f'Layer {len(attention_weights) + layer_idx}'
                axes[0, col + 1].set_title(f'{layer_name}\nCLS Attention Map', 
                                          fontsize=12, fontweight='bold')
                axes[0, col + 1].axis('off')
                plt.colorbar(im1, ax=axes[0, col + 1], fraction=0.046)
                
                # 2. Attention overlay on original image
                # Resize attention map to match image size
                attn_resized = torch.nn.functional.interpolate(
                    torch.tensor(attn_map).unsqueeze(0).unsqueeze(0), 
                    size=(img_display.shape[0], img_display.shape[1]), 
                    mode='bilinear'
                ).squeeze().numpy()
                
                axes[1, col + 1].imshow(img_display, cmap='gray' if len(img_display.shape) == 2 else None)
                axes[1, col + 1].imshow(attn_resized, alpha=0.6, cmap='jet', vmin=0, vmax=attn_resized.max())
                axes[1, col + 1].set_title(f'{layer_name}\nAttention Overlay', 
                                          fontsize=12, fontweight='bold')
                axes[1, col + 1].axis('off')
                
                # 3. Multi-head attention analysis
                num_heads = min(4, attn.shape[0])
                head_attns = []
                
                for head in range(num_heads):
                    head_attn = attn[head, 0, 1:].cpu().numpy().reshape(grid_size, grid_size)
                    head_attns.append(head_attn)
                
                # Create combined multi-head visualization
                combined_heads = np.concatenate(head_attns[:2], axis=1) if len(head_attns) >= 2 else head_attns[0]
                if len(head_attns) >= 4:
                    bottom_heads = np.concatenate(head_attns[2:4], axis=1)
                    combined_heads = np.concatenate([combined_heads, bottom_heads], axis=0)
                
                im3 = axes[2, col + 1].imshow(combined_heads, cmap='viridis')
                axes[2, col + 1].set_title(f'{layer_name}\nMulti-Head Attention\n(First {num_heads} heads)', 
                                          fontsize=12, fontweight='bold')
                axes[2, col + 1].axis('off')
                plt.colorbar(im3, ax=axes[2, col + 1], fraction=0.046)
        
        # Attention statistics in the leftmost column
        axes[1, 0].axis('off')
        axes[2, 0].axis('off')
        
        # Calculate attention statistics
        stats_text = "🔍 Attention Statistics:\n\n"
        
        for i, layer_idx in enumerate(layer_indices):
            if abs(layer_idx) > len(attention_weights):
                continue
                
            attn = attention_weights[layer_idx].squeeze(0)
            cls_attn = attn.mean(dim=0)[0, 1:]  # Average across heads, CLS to patches
            
            layer_name = f'Layer {layer_idx}' if layer_idx >= 0 else f'Layer {len(attention_weights) + layer_idx}'
            stats_text += f"{layer_name}:\n"
            stats_text += f"  Mean: {cls_attn.mean().item():.4f}\n"
            stats_text += f"  Std: {cls_attn.std().item():.4f}\n"
            stats_text += f"  Max: {cls_attn.max().item():.4f}\n"
            stats_text += f"  Entropy: {(-cls_attn * torch.log(cls_attn + 1e-8)).sum().item():.4f}\n\n"
        
        axes[1, 0].text(0.1, 0.9, stats_text, fontsize=11, verticalalignment='top',
                       transform=axes[1, 0].transAxes, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
        
        # Attention evolution analysis
        evolution_text = "📈 Attention Evolution:\n\n"
        evolution_text += "Early Layers:\n"
        evolution_text += "• Focus on local patterns\n"
        evolution_text += "• High attention entropy\n"
        evolution_text += "• Distributed attention\n\n"
        evolution_text += "Middle Layers:\n"
        evolution_text += "• Semantic grouping\n"
        evolution_text += "• Moderate specificity\n"
        evolution_text += "• Object-part relationships\n\n"
        evolution_text += "Later Layers:\n"
        evolution_text += "• Global object focus\n"
        evolution_text += "• Sharp attention peaks\n"
        evolution_text += "• Task-relevant regions\n\n"
        evolution_text += "Key Insights:\n"
        evolution_text += "• Hierarchical processing\n"
        evolution_text += "• Progressive specialization\n"
        evolution_text += "• Context aggregation"
        
        axes[2, 0].text(0.1, 0.9, evolution_text, fontsize=10, verticalalignment='top',
                       transform=axes[2, 0].transAxes,
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"💾 Vision Transformer attention visualization saved to: {save_path}")
        
        return cls_attn

def compare_attention_across_transformer_layers(model, sample_image, save_path):
    """Compare attention patterns across all transformer layers"""
    model.eval()
    
    with torch.no_grad():
        if sample_image.dim() == 3:
            sample_image = sample_image.unsqueeze(0)
        
        sample_image = sample_image.to(device)
        outputs, attention_weights = model(sample_image)
        
        if not attention_weights:
            print("⚠️ No attention weights available")
            return
        
        # Select layers to compare (every 2nd layer + first and last)
        layer_indices = [0, 2, 4, 6, 8, 10, -1]
        layer_indices = [i for i in layer_indices if i < len(attention_weights) or i == -1]
        
        fig, axes = plt.subplots(2, len(layer_indices), figsize=(4*len(layer_indices), 8))
        if len(layer_indices) == 1:
            axes = axes.reshape(2, 1)
        
        attention_evolution = []
        entropy_evolution = []
        
        for col, layer_idx in enumerate(layer_indices):
            actual_idx = layer_idx if layer_idx >= 0 else len(attention_weights) + layer_idx
            
            # Get attention weights
            attn = attention_weights[layer_idx].squeeze(0).mean(dim=0)  # Average across heads
            cls_attn = attn[0, 1:]  # CLS to patches attention
            
            # Calculate spatial dimensions
            num_patches = len(cls_attn)
            grid_size = int(np.sqrt(num_patches))
            
            if grid_size * grid_size == num_patches:
                attn_map = cls_attn.cpu().numpy().reshape(grid_size, grid_size)
                attention_evolution.append(attn_map)
                
                # Calculate attention entropy
                entropy = (-cls_attn * torch.log(cls_attn + 1e-8)).sum().item()
                entropy_evolution.append(entropy)
                
                # Plot attention map
                im1 = axes[0, col].imshow(attn_map, cmap='viridis', interpolation='bilinear')
                axes[0, col].set_title(f'Layer {actual_idx}\nEntropy: {entropy:.2f}', 
                                     fontsize=12, fontweight='bold')
                axes[0, col].axis('off')
                plt.colorbar(im1, ax=axes[0, col], fraction=0.046)
                
                # Plot attention distribution
                axes[1, col].hist(cls_attn.cpu().numpy(), bins=20, alpha=0.7, 
                                color='skyblue', edgecolor='black')
                axes[1, col].set_title(f'Attention Distribution\nLayer {actual_idx}', 
                                     fontsize=10, fontweight='bold')
                axes[1, col].set_xlabel('Attention Weight')
                axes[1, col].set_ylabel('Frequency')
                axes[1, col].grid(True, alpha=0.3)
        
        plt.suptitle('Attention Evolution Across Transformer Layers', 
                     fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Create entropy evolution plot
        if len(entropy_evolution) > 1:
            plt.figure(figsize=(10, 6))
            layer_numbers = [i if i >= 0 else len(attention_weights) + i for i in layer_indices]
            plt.plot(layer_numbers, entropy_evolution, 'o-', linewidth=3, markersize=8, color='darkblue')
            plt.title('Attention Entropy Evolution Through Transformer Layers', 
                     fontsize=14, fontweight='bold')
            plt.xlabel('Layer Index')
            plt.ylabel('Attention Entropy')
            plt.grid(True, alpha=0.3)
            
            # Add trend analysis
            if len(entropy_evolution) >= 3:
                z = np.polyfit(layer_numbers, entropy_evolution, 1)
                p = np.poly1d(z)
                plt.plot(layer_numbers, p(layer_numbers), "--", alpha=0.8, color='red', 
                        label=f'Trend: slope={z[0]:.3f}')
                plt.legend()
            
            entropy_save_path = save_path.replace('.png', '_entropy_evolution.png')
            plt.savefig(entropy_save_path, dpi=300, bbox_inches='tight')
            plt.show()
            
            print(f"💾 Attention entropy evolution saved to: {entropy_save_path}")
        
        print(f"💾 Layer comparison saved to: {save_path}")
        
        return attention_evolution

# Execute Vision Transformer attention analysis
if 'ViT-Tiny' in vit_models:
    print("\n👁️ Analyzing Vision Transformer Attention Mechanisms:")
    
    # Prepare sample image for attention analysis
    transform_vit_analysis = transforms.Compose([
        transforms.Resize(32),  # Match ViT input size
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    
    try:
        # Load a sample from CIFAR-10 test set
        test_dataset_vit = torchvision.datasets.CIFAR10(
            root='../data/modern_cnn_architectures/cifar10', 
            train=False, download=False, transform=transform_vit_analysis
        )
        sample_image_vit, sample_label_vit = test_dataset_vit[42]  # Fixed sample for consistency
        print(f"   Sample image class: {cifar10_classes[sample_label_vit]}")
        
    except:
        # Create dummy sample
        sample_image_vit = torch.randn(3, 32, 32)
        sample_label_vit = 0
        print("   Using dummy sample image")
    
    # Comprehensive attention visualization
    print("\n🔍 Creating comprehensive attention visualization...")
    visualize_vision_transformer_attention(
        vit_models['ViT-Tiny'],
        sample_image_vit,
        f"{project_dirs['attention_visualization']}/vit_comprehensive_attention_analysis.png"
    )
    
    # Layer-wise attention comparison
    print("\n📊 Comparing attention across transformer layers...")
    compare_attention_across_transformer_layers(
        vit_models['ViT-Tiny'],
        sample_image_vit,
        f"{project_dirs['attention_visualization']}/vit_layer_attention_comparison.png"
    )
    
else:
    print("⚠️ Vision Transformer model not available for attention analysis")

## 10. Final Results Summary and Model Comparison

In [ ]:
def create_comprehensive_results_summary():
    """Generate comprehensive summary of all experiments and analyses"""
    
    summary_data = {
        'experiment_info': {
            'title': 'Modern CNN Architectures: From ResNet to Vision Transformers',
            'completion_timestamp': datetime.now().isoformat(),
            'device_used': str(device),
            'pytorch_version': torch.__version__,
            'total_models_analyzed': len(models_info)
        },
        'architecture_families': {
            'ResNet': {
                'key_innovation': 'Skip connections and residual learning',
                'advantages': ['Deep network training', 'Gradient flow', 'Proven performance'],
                'models_implemented': list(resnet_models.keys()),
                'parameter_range': f"{min(models_info[k]['total_parameters'] for k in models_info if 'ResNet' in k):,} - {max(models_info[k]['total_parameters'] for k in models_info if 'ResNet' in k):,}"
            },
            'DenseNet': {
                'key_innovation': 'Dense connectivity and feature reuse',
                'advantages': ['Parameter efficiency', 'Feature reuse', 'Gradient flow'],
                'models_implemented': list(densenet_models.keys()),
                'parameter_range': f"{min(models_info[k]['total_parameters'] for k in models_info if 'DenseNet' in k):,} - {max(models_info[k]['total_parameters'] for k in models_info if 'DenseNet' in k):,}"
            },
            'EfficientNet': {
                'key_innovation': 'Compound scaling and mobile optimization',
                'advantages': ['Efficiency', 'Mobile deployment', 'Scalability'],
                'models_implemented': list(efficientnet_models.keys()),
                'parameter_range': f"{min(models_info[k]['total_parameters'] for k in models_info if 'EfficientNet' in k):,} - {max(models_info[k]['total_parameters'] for k in models_info if 'EfficientNet' in k):,}"
            },
            'Vision Transformer': {
                'key_innovation': 'Self-attention for computer vision',
                'advantages': ['Global context', 'Scalability', 'Transfer learning'],
                'models_implemented': list(vit_models.keys()),
                'parameter_range': f"{min(models_info[k]['total_parameters'] for k in models_info if 'ViT' in k):,} - {max(models_info[k]['total_parameters'] for k in models_info if 'ViT' in k):,}"
            }
        },
        'performance_analysis': performance_results if 'performance_results' in locals() else {},
        'transfer_learning_insights': {
            'strategies_tested': ['frozen', 'fine_tuning', 'progressive'],
            'models_evaluated': list(transfer_experiments.keys()) if 'transfer_experiments' in locals() else [],
            'key_findings': [
                'Fine-tuning generally achieves highest accuracy',
                'Frozen features provide fastest training',
                'Progressive unfreezing offers good balance',
                'Model architecture significantly impacts transfer success'
            ]
        },
        'technical_achievements': {
            'implemented_from_scratch': [
                'ResNet with skip connections',
                'DenseNet with dense connectivity', 
                'EfficientNet with compound scaling',
                'Vision Transformer with self-attention'
            ],
            'analysis_capabilities': [
                'Comprehensive architecture comparison',
                'Performance benchmarking',
                'Transfer learning experiments',
                'Attention mechanism visualization',
                'Feature extraction and analysis'
            ]
        },
        'model_recommendations': {
            'high_accuracy_large_datasets': ['ResNet-50', 'ViT-Small'],
            'mobile_deployment': ['EfficientNet-B0', 'EfficientNet-B1'],
            'memory_constrained': ['ResNet-18', 'EfficientNet-B0'],
            'research_experimentation': ['DenseNet-121', 'ViT-Tiny'],
            'transfer_learning': ['ResNet-18', 'EfficientNet-B0', 'ViT-Tiny']
        }
    }
    
    return summary_data

def create_final_dashboard(models_info, performance_results, save_path):
    """Create comprehensive final dashboard summarizing all analyses"""
    
    fig = plt.figure(figsize=(20, 24))
    gs = fig.add_gridspec(6, 4, hspace=0.3, wspace=0.3)
    
    # 1. Model Overview - Parameters and Size (Top Row)
    ax1 = fig.add_subplot(gs[0, :2])
    model_names = list(models_info.keys())
    param_counts = [info['total_parameters'] / 1e6 for info in models_info.values()]
    model_sizes = [info['model_size_mb'] for info in models_info.values()]
    
    # Color by family
    colors = []
    for name in model_names:
        if 'ResNet' in name: colors.append('#FF6B6B')
        elif 'DenseNet' in name: colors.append('#4ECDC4')
        elif 'EfficientNet' in name: colors.append('#45B7D1')
        elif 'ViT' in name: colors.append('#96CEB4')
        else: colors.append('#FFEAA7')
    
    bars1 = ax1.bar(range(len(model_names)), param_counts, color=colors, alpha=0.8)
    ax1.set_title('Model Parameter Count Comparison', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Parameters (Millions)', fontsize=12)
    ax1.set_xticks(range(len(model_names)))
    ax1.set_xticklabels(model_names, rotation=45, ha='right')
    
    # Add value labels
    for bar, count in zip(bars1, param_counts):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + max(param_counts)*0.01,
                f'{count:.1f}M', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 2. Performance Comparison (if available)
    ax2 = fig.add_subplot(gs[0, 2:])
    if performance_results:
        perf_models = list(performance_results.keys())
        inference_times = [performance_results[model]['avg_inference_time_ms'] for model in perf_models]
        throughputs = [performance_results[model]['throughput_fps'] for model in perf_models]
        
        ax2_twin = ax2.twinx()
        
        line1 = ax2.bar(range(len(perf_models)), inference_times, alpha=0.7, color='lightcoral', label='Inference Time (ms)')
        line2 = ax2_twin.plot(range(len(perf_models)), throughputs, 'o-', color='darkblue', linewidth=3, markersize=8, label='Throughput (FPS)')
        
        ax2.set_title('Performance Metrics Comparison', fontsize=16, fontweight='bold')
        ax2.set_ylabel('Inference Time (ms)', fontsize=12, color='darkred')
        ax2_twin.set_ylabel('Throughput (FPS)', fontsize=12, color='darkblue')
        ax2.set_xticks(range(len(perf_models)))
        ax2.set_xticklabels(perf_models, rotation=45, ha='right')
        
        # Combined legend
        lines1, labels1 = ax2.get_legend_handles_labels()
        lines2, labels2 = ax2_twin.get_legend_handles_labels()
        ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    else:
        ax2.text(0.5, 0.5, 'Performance Analysis\nNot Available', ha='center', va='center',
                transform=ax2.transAxes, fontsize=14, bbox=dict(boxstyle="round", facecolor='lightgray'))
        ax2.set_title('Performance Analysis', fontsize=16, fontweight='bold')
    
    # 3. Architecture Timeline (Second Row)
    ax3 = fig.add_subplot(gs[1, :2])
    innovation_timeline = {
        'AlexNet': 2012, 'VGG': 2014, 'ResNet': 2015, 'DenseNet': 2016, 
        'MobileNet': 2017, 'EfficientNet': 2019, 'Vision Transformer': 2020, 'ConvNeXt': 2022
    }
    
    timeline_years = list(innovation_timeline.values())
    timeline_names = list(innovation_timeline.keys())
    implemented_models = ['ResNet', 'DenseNet', 'EfficientNet', 'Vision Transformer']
    
    colors_timeline = ['red' if name in implemented_models else 'lightgray' for name in timeline_names]
    sizes = [150 if name in implemented_models else 50 for name in timeline_names]
    
    scatter = ax3.scatter(timeline_years, range(len(timeline_names)), c=colors_timeline, s=sizes, alpha=0.8)
    for i, (name, year) in enumerate(innovation_timeline.items()):
        weight = 'bold' if name in implemented_models else 'normal'
        ax3.annotate(name, (year, i), xytext=(10, 0), textcoords='offset points',
                    fontsize=11, fontweight=weight)
    
    ax3.set_title('CNN Architecture Innovation Timeline', fontsize=16, fontweight='bold')
    ax3.set_xlabel('Year')
    ax3.set_yticks(range(len(timeline_names)))
    ax3.set_yticklabels(timeline_names)
    ax3.grid(True, alpha=0.3)
    ax3.text(0.02, 0.98, 'Red: Implemented\nGray: Reference', transform=ax3.transAxes, 
             va='top', bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
    
    # 4. Key Innovations Summary
    ax4 = fig.add_subplot(gs[1, 2:])
    ax4.axis('off')
    
    innovations_text = """
🔗 ResNet (2015): Skip Connections
   • Solves vanishing gradient problem
   • Enables very deep networks (50-152 layers)
   • Residual learning framework

🌟 DenseNet (2016): Dense Connectivity  
   • Every layer connects to every other layer
   • Maximum information flow between layers
   • Parameter efficient architecture

⚡ EfficientNet (2019): Compound Scaling
   • Systematically scales depth, width, resolution
   • Mobile-optimized with SE blocks
   • Best accuracy-efficiency trade-offs

👁️ Vision Transformer (2020): Self-Attention
   • Applies transformer to computer vision
   • Global context through attention
   • Patch-based image processing
    """
    
    ax4.text(0.05, 0.95, innovations_text, fontsize=11, verticalalignment='top',
             transform=ax4.transAxes)
    ax4.set_title('Key Architecture Innovations', fontsize=16, fontweight='bold')
    
    # 5. Transfer Learning Results (if available)
    ax5 = fig.add_subplot(gs[2, :])
    if 'transfer_experiments' in locals() and transfer_experiments:
        # Create transfer learning summary
        strategies = ['frozen', 'fine_tuning', 'progressive']
        models_tl = list(transfer_experiments.keys())
        
        # Create heatmap-style visualization
        tl_data = []
        for model in models_tl:
            row = []
            for strategy in strategies:
                if strategy in transfer_experiments[model].results:
                    final_acc = transfer_experiments[model].results[strategy]['test_acc'][-1]
                    row.append(final_acc)
                else:
                    row.append(np.nan)
            tl_data.append(row)
        
        im = ax5.imshow(tl_data, cmap='RdYlGn', aspect='auto', vmin=60, vmax=90)
        ax5.set_title('Transfer Learning Results: Test Accuracy (%)', fontsize=16, fontweight='bold')
        ax5.set_xticks(range(len(strategies)))
        ax5.set_xticklabels([s.replace('_', ' ').title() for s in strategies])
        ax5.set_yticks(range(len(models_tl)))
        ax5.set_yticklabels(models_tl)
        
        # Add text annotations
        for i in range(len(models_tl)):
            for j in range(len(strategies)):
                if not np.isnan(tl_data[i][j]):
                    text = ax5.text(j, i, f'{tl_data[i][j]:.1f}%',
                                   ha="center", va="center", color="black", fontweight='bold')
        
        plt.colorbar(im, ax=ax5, label='Test Accuracy (%)')
    else:
        ax5.text(0.5, 0.5, 'Transfer Learning Analysis\nNot Available', ha='center', va='center',
                transform=ax5.transAxes, fontsize=14, bbox=dict(boxstyle="round", facecolor='lightgray'))
        ax5.set_title('Transfer Learning Analysis', fontsize=16, fontweight='bold')
    
    # 6. Model Selection Guide
    ax6 = fig.add_subplot(gs[3, :2])
    ax6.axis('off')
    
    selection_guide = """
🎯 Model Selection Guide:

📱 Mobile & Edge Deployment:
   → EfficientNet-B0: Best efficiency
   → MobileNet variants: Ultra-lightweight
   
🏆 High Accuracy Applications:
   → ResNet-50: Proven performance
   → ViT-Small: State-of-the-art accuracy
   
💾 Memory Constrained:
   → ResNet-18: Balanced choice
   → EfficientNet-B0: Optimized efficiency
   
🔬 Research & Experimentation:
   → DenseNet-121: Novel connectivity
   → ViT-Tiny: Attention mechanisms
   
🎯 Transfer Learning:
   → ResNet-18: Versatile backbone
   → EfficientNet-B0: Efficient transfer
    """
    
    ax6.text(0.05, 0.95, selection_guide, fontsize=11, verticalalignment='top',
             transform=ax6.transAxes)
    ax6.set_title('Model Selection Guidelines', fontsize=16, fontweight='bold')
    
    # 7. Performance Trade-offs
    ax7 = fig.add_subplot(gs[3, 2:])
    ax7.axis('off')
    
    tradeoffs_text = """
⚖️ Architecture Trade-offs:

🎯 Accuracy vs Efficiency:
   High Accuracy: ViT > ResNet > DenseNet > EfficientNet
   High Efficiency: EfficientNet > ResNet > DenseNet > ViT

⚡ Speed vs Quality:
   Fast Inference: EfficientNet > ResNet > DenseNet > ViT
   High Quality: ViT > ResNet > DenseNet > EfficientNet

💾 Memory vs Performance:
   Low Memory: EfficientNet > ResNet > ViT > DenseNet
   High Performance: ViT > ResNet > DenseNet > EfficientNet

🔧 Training vs Inference:
   Training Efficient: ResNet (balanced across metrics)
   Inference Optimized: EfficientNet (mobile-first design)

📱 Mobile vs Desktop:
   Mobile Optimized: EfficientNet series
   Desktop/Server: ResNet, ViT series
    """
    
    ax7.text(0.05, 0.95, tradeoffs_text, fontsize=11, verticalalignment='top',
             transform=ax7.transAxes)
    ax7.set_title('Performance Trade-offs Analysis', fontsize=16, fontweight='bold')
    
    # 8. Key Findings and Insights
    ax8 = fig.add_subplot(gs[4, :])
    ax8.axis('off')
    
    findings_text = """
🔍 Key Findings and Insights:

📊 Architecture Comparison:
   • ResNet family provides excellent balance of performance, training stability, and computational efficiency
   • DenseNet achieves high parameter efficiency but requires significant memory for dense connections
   • EfficientNet demonstrates superior efficiency through compound scaling and mobile optimization
   • Vision Transformers show exceptional scalability and transfer learning capabilities

🎯 Transfer Learning Insights:
   • Fine-tuning consistently outperforms feature extraction for sufficient data scenarios
   • Progressive unfreezing offers compromise between computational cost and performance
   • Architecture choice significantly impacts transfer learning success rates
   • Pretrained weights provide substantial performance boost across all architectures

⚡ Performance Characteristics:
   • EfficientNet variants achieve best inference speed for mobile deployment scenarios
   • ResNet architectures provide most consistent performance across different hardware configurations
   • Vision Transformers require significant computational resources but offer superior global context modeling
   • DenseNet models show excellent feature reuse but suffer from memory bottlenecks during training

💡 Practical Recommendations:
   • Use EfficientNet for mobile applications requiring real-time performance
   • Choose ResNet for general-purpose applications requiring proven stability
   • Consider Vision Transformers for tasks requiring global context understanding
   • Apply DenseNet when parameter efficiency is critical constraint
    """
    
    ax8.text(0.02, 0.95, findings_text, fontsize=11, verticalalignment='top',
             transform=ax8.transAxes)
    ax8.set_title('Key Findings and Practical Insights', fontsize=18, fontweight='bold')
    
    # 9. Future Directions
    ax9 = fig.add_subplot(gs[5, :])
    ax9.axis('off')
    
    future_text = """
🚀 Future Directions and Next Steps:

🔬 Advanced Architecture Exploration:
   • ConvNeXt: Modernized CNN design inspired by Vision Transformers
   • Swin Transformer: Hierarchical attention for computer vision
   • RegNet: Design space exploration for optimal architectures
   • NFNet: Normalizer-free networks for improved training

🎯 Specialized Applications:
   • Object Detection: YOLO, R-CNN family integration
   • Semantic Segmentation: UNet, DeepLab adaptations  
   • Video Understanding: 3D convolutions and temporal modeling
   • Multi-modal Learning: CLIP-style vision-language models

⚡ Optimization and Deployment:
   • Model Compression: Pruning, quantization, knowledge distillation
   • Neural Architecture Search: Automated design optimization
   • Edge Deployment: TensorRT, ONNX optimization pipelines
   • Distributed Training: Multi-GPU and multi-node scaling

🔧 Advanced Training Techniques:
   • Self-supervised Learning: MAE, SimCLR, DINO approaches
   • Meta-learning: Few-shot adaptation strategies
   • Continual Learning: Avoiding catastrophic forgetting
   • Adversarial Robustness: Defending against attacks

💾 Implementation Improvements:
   • Mixed Precision Training: FP16/BF16 acceleration
   • Gradient Checkpointing: Memory-efficient training
   • Dynamic Batching: Adaptive computational graphs
   • Custom CUDA Kernels: Hardware-specific optimizations
    """
    
    ax9.text(0.02, 0.95, future_text, fontsize=11, verticalalignment='top',
             transform=ax9.transAxes)
    ax9.set_title('Future Directions and Advanced Topics', fontsize=18, fontweight='bold')
    
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"💾 Comprehensive final dashboard saved to: {save_path}")

# Generate comprehensive results summary
print("\n📋 Generating Comprehensive Results Summary...")
final_summary = create_comprehensive_results_summary()

# Save comprehensive summary to JSON
summary_save_path = f"{project_dirs['analysis']}/comprehensive_project_summary.json"
with open(summary_save_path, 'w') as f:
    json.dump(final_summary, f, indent=2, default=str)

print(f"💾 Project summary saved to: {summary_save_path}")

# Create final comprehensive dashboard
print("\n🎨 Creating Final Comprehensive Dashboard...")
create_final_dashboard(
    models_info,
    performance_results if 'performance_results' in locals() else {},
    f"{project_dirs['analysis']}/final_comprehensive_dashboard.png"
)

# Save all model states
print("\n💾 Saving Model Checkpoints...")
models_to_save = {**resnet_models, **densenet_models, **efficientnet_models, **vit_models}

for model_name, model in models_to_save.items():
    checkpoint_path = f"{project_dirs['final']}/{model_name.lower().replace('-', '_')}_final.pth"
    
    checkpoint_data = {
        'model_state_dict': model.state_dict(),
        'model_info': models_info[model_name],
        'architecture_type': model_name.split('-')[0],
        'timestamp': datetime.now().isoformat(),
        'pytorch_version': torch.__version__,
        'device_trained': str(device)
    }
    
    torch.save(checkpoint_data, checkpoint_path)
    print(f"   ✅ {model_name} saved to {checkpoint_path}")

print("\n" + "="*80)
print("🎉 MODERN CNN ARCHITECTURES PROJECT COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n🏆 Project Achievements:")
print(f"   ✅ {len(models_info)} architecture variants implemented from scratch")
print(f"   ✅ 4 major architecture families covered (ResNet, DenseNet, EfficientNet, ViT)")
print(f"   ✅ Comprehensive performance benchmarking completed")
print(f"   ✅ Advanced transfer learning experiments executed")
print(f"   ✅ Vision Transformer attention mechanisms analyzed")
print(f"   ✅ Professional visualizations and analysis reports generated")

print(f"\n📊 Technical Highlights:")
print(f"   🔗 ResNet: Skip connections and deep network training")
print(f"   🌟 DenseNet: Dense connectivity and feature reuse")
print(f"   ⚡ EfficientNet: Compound scaling and mobile optimization") 
print(f"   👁️ ViT: Self-attention and global context modeling")

print(f"\n📁 Generated Deliverables:")
print(f"   🎨 Comprehensive architecture analysis and comparisons")
print(f"   ⚡ Performance benchmarking and efficiency analysis")
print(f"   🎯 Transfer learning experimental results")
print(f"   👁️ Vision Transformer attention visualizations")
print(f"   💾 Complete model implementations and checkpoints")
print(f"   📋 Detailed analysis reports and recommendations")

print(f"\n🚀 Ready for Advanced Topics:")
print(f"   • Object Detection and Segmentation")
print(f"   • Neural Architecture Search")
print(f"   • Model Compression and Optimization")
print(f"   • Multi-modal Learning")
print(f"   • Production Deployment Strategies")

print(f"\n📚 All materials saved to: ../results/modern_cnn_architectures/")
print("="*80)